In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
import os
from typing import List

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold
from sklearn.cluster import KMeans
from multiprocessing import Pool as MultiprocessingPool, cpu_count

from src.kinematics import calculate_speed_and_direction

pd.set_option("display.max_columns", None)

/Users/kniu91/Documents/projects/bdb-26/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ============================================================================
# CONFIG
# ============================================================================

class Config:
    DATA_DIR = Path("./data")
    OUTPUT_DIR = Path("./outputs")
    OUTPUT_DIR.mkdir(exist_ok=True)
    
    SEED = 42
    N_FOLDS = 5
    BATCH_SIZE = 256
    EPOCHS = 60
    PATIENCE = 30
    LEARNING_RATE = 1e-4
    
    WINDOW_SIZE = 10
    HIDDEN_DIM = 128
    MAX_FUTURE_HORIZON = 94
    
    FIELD_X_MIN, FIELD_X_MAX = 0.0, 120.0
    FIELD_Y_MIN, FIELD_Y_MAX = 0.0, 53.3
    
    K_NEIGH = 6
    RADIUS = 30.0
    TAU = 8.0
    N_ROUTE_CLUSTERS = 7
    
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def set_seed(seed=42):
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seed(Config.SEED)



In [3]:
config = Config()
config

print("\n[1/4] Loading data...")
train_input_files = [config.DATA_DIR / f"train/input_2023_w{w:02d}.csv" for w in range(1, 19)]
train_output_files = [config.DATA_DIR / f"train/output_2023_w{w:02d}.csv" for w in range(1, 19)]
train_input = pd.concat([pd.read_csv(f) for f in train_input_files if f.exists()])
train_output = pd.concat([pd.read_csv(f) for f in train_output_files if f.exists()])
supplementary_data = pd.read_csv(config.DATA_DIR / "supplementary_data.csv")

print(f"✓ Train input: {train_input.shape}, Train output: {train_output.shape}")
print(f"✓ Train output: {train_output.shape}, unique plays: {train_output[['game_id','play_id']].drop_duplicates().shape[0]}")
print(f"✓ Supplementary data: {supplementary_data.shape}")

traj_output = pd.read_csv('local_submission.csv')
traj_output = traj_output[['game_id', 'play_id', 'nfl_id', 'frame_id', 'pred_x', 'pred_y']]
traj_output.rename(columns={'pred_x': 'x', 'pred_y': 'y'}, inplace=True)

train_output.sort_values(by=['game_id', 'play_id', 'nfl_id', 'frame_id'], inplace=True)
traj_output.sort_values(by=['game_id', 'play_id', 'nfl_id', 'frame_id'], inplace=True)

print(f"✓ Projected trajectory output: {traj_output.shape}, unique plays: {traj_output[['game_id','play_id']].drop_duplicates().shape[0]}")


[1/4] Loading data...


/var/folders/_m/rvnpg_cs6xzcz0vlml0lzkbm0000gp/T/ipykernel_39940/1380030048.py:9: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  supplementary_data = pd.read_csv(config.DATA_DIR / "supplementary_data.csv")


✓ Train input: (4880579, 23), Train output: (562936, 6)
✓ Train output: (562936, 6), unique plays: 14108
✓ Supplementary data: (18009, 41)
✓ Projected trajectory output: (562936, 6), unique plays: 14108


In [4]:
local_submission = pd.read_csv('local_submission.csv')
a = train_input[['game_id','play_id','nfl_id','player_role']].drop_duplicates()
labeled_output = train_output.merge(a, on=['game_id','play_id','nfl_id'], how='left')
print(labeled_output.shape)
labeled_output.head()

keys = ['game_id','play_id','nfl_id','frame_id']
loc_sub_cols = keys + ['pred_x','pred_y']
labeled_output = labeled_output.merge(local_submission[loc_sub_cols], on=['game_id','play_id','nfl_id','frame_id'], how='left', suffixes=('_true','_pred'))
labeled_output['error'] = np.sqrt((labeled_output['x'] - labeled_output['pred_x'])**2 + (labeled_output['y'] - labeled_output['pred_y'])**2)
labeled_output['error_diff'] = labeled_output.groupby(['game_id', 'play_id', 'nfl_id'])['error'].diff()
labeled_output.head(20)

receiver_positions = labeled_output.loc[
  labeled_output['player_role'] == 'Targeted Receiver',
  ['game_id','play_id','nfl_id', 'frame_id','x','y']].drop_duplicates()
receiver_positions.rename(columns = {'x':'target_x', 'y':'target_y'}, inplace = True)
print(receiver_positions.shape)
receiver_positions.head(3)

labeled_output = labeled_output.merge(
    receiver_positions[['game_id','play_id','frame_id','target_x','target_y']],
    on=['game_id','play_id','frame_id'], how='left'
)
print(labeled_output.shape)
labeled_output.head(3)

labeled_output['distance_from_receiver'] = np.sqrt(
    (labeled_output['x'] - labeled_output['target_x'])**2 +
    (labeled_output['y'] - labeled_output['target_y'])**2
)

# labeled_output['distance_from_receiver_l1'] = labeled_output.groupby(['game_id', 'play_id', 'nfl_id'])['distance_from_receiver'].shift(1)
labeled_output['distance_from_receiver_l3'] = labeled_output.groupby(['game_id', 'play_id', 'nfl_id'])['distance_from_receiver'].shift(3)
labeled_output['distance_from_receiver_l5'] = labeled_output.groupby(['game_id', 'play_id', 'nfl_id'])['distance_from_receiver'].shift(5)
labeled_output['distance_from_receiver_l7'] = labeled_output.groupby(['game_id', 'play_id', 'nfl_id'])['distance_from_receiver'].shift(7)

labeled_output['distance_from_receiver_min_l7'] = labeled_output[[
                                                                # 'distance_from_receiver',
                                                                # 'distance_from_receiver_l1',
                                                                'distance_from_receiver_l3',
                                                                'distance_from_receiver_l5',
                                                                'distance_from_receiver_l7']].min(axis=1) 
labeled_output[
  (labeled_output['game_id'] == 2023091013) &
  (labeled_output['play_id'] == 3686) &
  (labeled_output['nfl_id'] == 54632)
]

# test = labeled_output[labeled_output['player_role'].str.contains('Defensive Coverage')]

(562936, 7)
(160360, 6)
(562936, 13)


,game_id,play_id,nfl_id,frame_id,x,y,player_role,pred_x,pred_y,error,error_diff,target_x,target_y,distance_from_receiver,distance_from_receiver_l3,distance_from_receiver_l5,distance_from_receiver_l7,distance_from_receiver_min_l7
29621,2023091013,3686,54632,1,46.63,39.84,Defensive Coverage,46.631927,39.860932,0.021020,NaN,43.87,40.61,2.865397,NaN,NaN,NaN,NaN
29622,2023091013,3686,54632,2,46.44,40.36,Defensive Coverage,46.512623,40.263906,0.120450,0.099430,43.89,41.11,2.658007,NaN,NaN,NaN,NaN
29623,2023091013,3686,54632,3,46.22,40.96,Defensive Coverage,46.370215,40.694341,0.305187,0.184738,43.88,41.61,2.428600,NaN,NaN,NaN,NaN
29624,2023091013,3686,54632,4,46.02,41.48,Defensive Coverage,46.206613,41.154565,0.375143,0.069956,43.84,42.13,2.274841,2.865397,NaN,NaN,2.865397
29625,2023091013,3686,54632,5,45.79,42.05,Defensive Coverage,46.025641,41.638732,0.473992,0.098849,43.77,42.64,2.104400,2.658007,NaN,NaN,2.658007
29626,2023091013,3686,54632,6,45.58,42.56,Defensive Coverage,45.828163,42.145513,0.483099,0.009107,43.68,43.15,1.989497,2.428600,2.865397,NaN,2.428600
29627,2023091013,3686,54632,7,45.29,43.15,Defensive Coverage,45.614781,42.675000,0.575419,0.092321,43.58,43.66,1.784433,2.274841,2.658007,NaN,2.274841
29628,2023091013,3686,54632,8,45.08,43.63,Defensive Coverage,45.398282,43.220472,0.518668,-0.056751,43.46,44.18,1.710819,2.104400,2.428600,2.865397,2.104400
29629,2023091013,3686,54632,9,44.67,44.28,Defensive Coverage,45.166847,43.777821,0.706428,0.187759,43.30,44.73,1.442012,1.989497,2.274841,2.658007,1.989497
29630,2023091013,3686,54632,10,43.99,45.14,Defensive Coverage,44.927816,44.349732,1.226386,0.519958,43.16,45.23,0.834865,1.784433,2.104400,2.428600,1.784433


In [5]:
labeled_output['potential_contact'] = np.where(
  (
    (labeled_output['player_role'] == 'Defensive Coverage') & 
    (labeled_output['error_diff'] > 0.1) & 
    (labeled_output['distance_from_receiver_min_l7'] < 1.25)
  ),
  1, 0
)


potential_contact = labeled_output.groupby(['game_id','play_id','nfl_id']).max()['potential_contact'].reset_index()
potential_contact['potential_contact'].value_counts()

potential_contact
0    44540
1     1505
Name: count, dtype: int64

In [6]:
# Get max frame_id for each player in each play (no warnings)
final_positions = (
    train_input
    .sort_values('frame_id')
    .groupby(['game_id', 'play_id', 'nfl_id'], as_index=False)
    .last()
    [['game_id', 'play_id', 'nfl_id', 'frame_id', 'x', 'y', 'player_role', 'ball_land_x', 'ball_land_y']]
)

# Calculate angle to ball landing spot
final_positions['angle_to_ball'] = np.degrees(np.arctan2(
    final_positions['ball_land_y'] - final_positions['y'],
    final_positions['ball_land_x'] - final_positions['x']
))

# Distance to ball
final_positions['distance_to_ball'] = np.sqrt(
    (final_positions['ball_land_x'] - final_positions['x'])**2 + 
    (final_positions['ball_land_y'] - final_positions['y'])**2
)

# Get targeted receiver positions AND angle
targeted_receivers = final_positions[
    final_positions['player_role'] == 'Targeted Receiver'
][['game_id', 'play_id', 'x', 'y', 'angle_to_ball']].rename(
    columns={
        'x': 'target_x', 
        'y': 'target_y',
        'angle_to_ball': 'target_angle_to_ball'
    }
)

# Merge back to get targeted receiver position for each play
final_positions = final_positions.merge(
    targeted_receivers,
    on=['game_id', 'play_id'],
    how='left'
)

# Calculate distance to targeted receiver
final_positions['distance_to_target_receiver'] = np.sqrt(
    (final_positions['target_x'] - final_positions['x'])**2 + 
    (final_positions['target_y'] - final_positions['y'])**2
)

# Calculate angle difference from targeted receiver
# Handle angle wrapping (e.g., difference between 359° and 1° should be 2°, not 358°)
angle_diff = final_positions['angle_to_ball'] - final_positions['target_angle_to_ball']
final_positions['angle_diff_from_target'] = np.where(
    angle_diff > 180, 
    angle_diff - 360,
    np.where(angle_diff < -180, angle_diff + 360, angle_diff)
)

# Absolute angle difference (unsigned)
final_positions['abs_angle_diff_from_target'] = np.abs(final_positions['angle_diff_from_target'])

# Optional: drop intermediate columns
final_positions = final_positions.drop(columns=['target_x', 'target_y', 'target_angle_to_ball'])

final_positions['defender_type'] = np.where((final_positions['distance_to_target_receiver'] < 6.0) &
                                            (final_positions['player_role'].str.contains('Defensive Coverage')) &
                                            (final_positions['abs_angle_diff_from_target'] < 90.0)
                                            , 'Trailing', np.where((final_positions['player_role'].str.contains('Defensive Coverage'))
                                                                   , 'Converging', None))

final_positions.sort_values(['game_id','play_id','nfl_id'], inplace=True)
print(f"Shape: {final_positions.shape}")
final_positions.head(3)

Shape: (173150, 15)


,game_id,play_id,nfl_id,frame_id,x,y,player_role,ball_land_x,ball_land_y,angle_to_ball,distance_to_ball,distance_to_target_receiver,angle_diff_from_target,abs_angle_diff_from_target,defender_type
0,2023090700,101,43290,26,35.41,29.99,Passer,63.259998,-0.22,-47.327657,41.088520,23.257319,5.649543,5.649543,None
1,2023090700,101,44930,26,52.43,14.14,Targeted Receiver,63.259998,-0.22,-52.977199,17.986063,0.000000,0.000000,0.000000,None
2,2023090700,101,46137,26,55.82,17.67,Defensive Coverage,63.259998,-0.22,-67.418810,19.375388,4.894180,-14.441611,14.441611,Trailing


In [7]:
final_positions.rename(columns={'x': 'throw_x', 'y': 'throw_y'}, inplace=True)
defender_type = train_output[['game_id','play_id','nfl_id']].drop_duplicates().merge(
    final_positions[['game_id','play_id','nfl_id','player_role','angle_to_ball','throw_x','throw_y','distance_to_ball', 'distance_to_target_receiver', 'abs_angle_diff_from_target', 'defender_type']],
    on=['game_id','play_id','nfl_id'], how='left'
)
print(defender_type.shape)
defender_type.tail(3)

(46045, 11)


,game_id,play_id,nfl_id,player_role,angle_to_ball,throw_x,throw_y,distance_to_ball,distance_to_target_receiver,abs_angle_diff_from_target,defender_type
46042,2024010713,4018,47844,Defensive Coverage,-80.352611,30.69,15.24,8.652364,8.013588,40.579130,Converging
46043,2024010713,4018,52457,Targeted Receiver,-120.931741,38.45,17.24,12.275871,0.000000,0.000000,None
46044,2024010713,4018,52647,Defensive Coverage,-152.507915,44.40,13.09,13.820710,7.254309,31.576174,Converging


In [8]:
def map_route_family(route: str) -> str:
    """
    For field named: route_type_of_targeted_receiver
    """
    if pd.isna(route):
        return "UNKNOWN"
    route = str(route).upper()
    
    vertical = {"GO", "POST", "CORNER", "WHEEL"}
    out_break = {"OUT", "FLAT"}
    in_break = {"IN", "SLANT", "CROSS", "ANGLE"}
    stop = {"HITCH"}
    screen = {"SCREEN"}
    
    if route in vertical:
        return "VERTICAL"
    if route in out_break:
        return "OUT_BREAK"
    if route in in_break:
        return "IN_BREAK"
    if route in stop:
        return "STOP_HITCH"
    if route in screen:
        return "SCREEN"
    return "OTHER"

def classify_down_distance(down, distance):
    """
    Classify down/distance into meaningful football situations.

    Returns compact labels like '3rd_long', '1st_short', etc.
    """
    if pd.isna(down) or pd.isna(distance):
        return "unknown"

    # Distance buckets
    if distance <= 3:
        dist_label = "short"
    elif distance <= 7:
        dist_label = "medium"
    else:
        dist_label = "long"

    # Special cases
    if down == 3 and distance >= 10:
        return "3rd_long"  # Most predictive situation
    elif down == 4:
        return "4th_down"  # All 4th downs are high-leverage
    elif down == 1 and distance <= 3:
        return "1st_short"  # Run-heavy
    elif down == 2 and distance >= 10:
        return "2nd_long"  # Pass-heavy
    else:
        return (
            f'{int(down)}{"st" if down==1 else "nd" if down==2 else "rd"}_{dist_label}'
        )


def bin_pass_length(pl):
    """
    pl: numeric pass_length (air yards)
    Bins: SHORT (0-10), INTERMEDIATE (10-20), DEEP (20+)
    """
    if pd.isna(pl):
        return "UNKNOWN"
    try:
        v = float(pl)
    except Exception:
        return "UNKNOWN"
    
    if v <= 10:
        return "SHORT"
    elif v <= 20:
        return "INTERMEDIATE"
    else:
        return "DEEP"

def prep_supplementary_data_for_icc(supplementary_data:pd.DataFrame) -> pd.DataFrame:
    supplementary_data["down_and_distance"] = supplementary_data.apply(
        lambda row: classify_down_distance(row["down"], row["yards_to_go"]), axis=1
    )

    supplementary_data["route_type_of_targeted_receiver"] = supplementary_data[
        "route_of_targeted_receiver"
    ].apply(map_route_family)

    supplementary_data["pass_length_bin"] = supplementary_data["pass_length"].apply(
        bin_pass_length
    )


    
    suppl_data_pre = supplementary_data[
        [
            "game_id",
            "play_id",
            "possession_team",
            "defensive_team",
            "down_and_distance",
            "receiver_alignment",
            "route_of_targeted_receiver",
            "route_type_of_targeted_receiver",
            "pass_length",
            "pass_length_bin",
            "dropback_type",
            # 'dropback_distance',
            "team_coverage_man_zone",
            "team_coverage_type",
            # "pass_result"
        ]
    ]
    return suppl_data_pre
suppl_data_pre = prep_supplementary_data_for_icc(supplementary_data)
print(suppl_data_pre.shape)
suppl_data_pre.head(3)

(18009, 13)


,game_id,play_id,possession_team,defensive_team,down_and_distance,receiver_alignment,route_of_targeted_receiver,route_type_of_targeted_receiver,pass_length,pass_length_bin,dropback_type,team_coverage_man_zone,team_coverage_type
0,2023090700,3461,DET,KC,3rd_long,3x2,IN,IN_BREAK,18,INTERMEDIATE,TRADITIONAL,ZONE_COVERAGE,COVER_2_ZONE
1,2023090700,461,DET,KC,1st_long,3x1,POST,VERTICAL,13,INTERMEDIATE,TRADITIONAL,ZONE_COVERAGE,COVER_6_ZONE
2,2023090700,1940,DET,KC,2nd_long,3x1,OUT,OUT_BREAK,18,INTERMEDIATE,TRADITIONAL,ZONE_COVERAGE,COVER_2_ZONE


In [9]:
suppl_data_pre['pass_length_bin'].value_counts()

pass_length_bin
SHORT           12089
INTERMEDIATE     3885
DEEP             2035
Name: count, dtype: int64

In [10]:
def bin_air_frames(n_frames):
    """
    n_frames: number of output frames while ball is in the air
    
    Bins by air time:
      QUICK  : <= 10 frames
      MEDIUM : 11-20 frames
      LONG   : > 20 frames
    """
    if pd.isna(n_frames):
        return "UNKNOWN"
    try:
        f = float(n_frames)
    except Exception:
        return "UNKNOWN"
    
    if f <= 10:
        return "QUICK"
    elif f <= 20:
        return "MEDIUM"
    else:
        return "LONG"

def prep_player_level_data_for_icc(
    train_input: pd.DataFrame, train_output: pd.DataFrame
) -> pd.DataFrame:
    qbs = train_input.loc[
        train_input["player_role"] == "Passer", ["game_id", "play_id", "nfl_id"]
    ].drop_duplicates()
    qbs = qbs.rename(columns={"nfl_id": "passer_nfl_id"})

    output_frame_count = (
        train_output.groupby(["game_id", "play_id"])["frame_id"].max().reset_index()
    )
    output_frame_count = output_frame_count.rename(
        columns={"frame_id": "num_output_frames"}
    )
    output_frame_count["num_output_frames_bin"] = output_frame_count[
        "num_output_frames"
    ].apply(bin_air_frames)

    train_input_pre = train_input[
        ["game_id", "play_id", "nfl_id", "player_position"]
    ].drop_duplicates()
    train_input_pre = train_input_pre.merge(qbs, on=["game_id", "play_id"], how="left")
    train_input_pre = train_input_pre.merge(
        output_frame_count, on=["game_id", "play_id"], how="left"
    )
    return train_input_pre

train_input_pre = prep_player_level_data_for_icc(train_input, train_output)
print(train_input_pre.shape)
train_input_pre.head(3)

(173150, 7)


,game_id,play_id,nfl_id,player_position,passer_nfl_id,num_output_frames,num_output_frames_bin
0,2023090700,101,54527,FS,43290.0,21,LONG
1,2023090700,101,46137,SS,43290.0,21,LONG
2,2023090700,101,52546,CB,43290.0,21,LONG


In [11]:
player_impacts = pd.read_csv('./outputs/defender_impact_lgb.csv')
print(player_impacts.shape)
player_impacts = player_impacts.merge(supplementary_data[['game_id','play_id','pass_result', 'yards_gained', 'season','week','home_team_abbr','visitor_team_abbr','play_description', 'quarter','game_clock']],
        on=['game_id','play_id'], how='left')
print(player_impacts.shape)
player_impacts = player_impacts.merge(train_input_pre,
        on=['game_id','play_id','nfl_id'], how='left')
print(player_impacts.shape)
player_impacts = player_impacts.merge(suppl_data_pre,
        on=['game_id','play_id'], how='left')
print(player_impacts.shape)

player_impacts = player_impacts.merge(defender_type[['game_id','play_id','nfl_id','defender_type']],
                                      on = ['game_id','play_id','nfl_id'],
                                      how = 'left')
print(player_impacts.shape)

player_impacts = player_impacts.merge(potential_contact[['game_id','play_id','nfl_id', 'potential_contact']],
                                      on = ['game_id','play_id','nfl_id'],
                                      how = 'left')
print(player_impacts.shape)

recs = player_impacts.loc[
  player_impacts['player_role'] == 'Targeted Receiver',
  ['game_id','play_id','nfl_id']].drop_duplicates().rename(columns = {'nfl_id':'targeted_receiver_id'})

player_impacts = player_impacts.merge(recs,
                                      on= ['game_id','play_id'],
                                      how = 'left')
print(player_impacts.shape)

player_impacts.sort_values('delta', ascending = False).head(3)

(44032, 8)
(44032, 17)
(44032, 21)
(44032, 32)
(44032, 33)
(44032, 34)
(44032, 35)


,game_id,play_id,nfl_id,player_role,baseline_prob,real_prob,delta,fold,pass_result,yards_gained,season,week,home_team_abbr,visitor_team_abbr,play_description,quarter,game_clock,player_position,passer_nfl_id,num_output_frames,num_output_frames_bin,possession_team,defensive_team,down_and_distance,receiver_alignment,route_of_targeted_receiver,route_type_of_targeted_receiver,pass_length,pass_length_bin,dropback_type,team_coverage_man_zone,team_coverage_type,defender_type,potential_contact,targeted_receiver_id
36793,2023100807,2356,53434,Targeted Receiver,0.055341,0.853926,0.798585,5,C,63,2023,5,ARI,CIN,(13:47) (Shotgun) J.Burrow pass deep right to ...,3,13:47,WR,52409.0,29,LONG,CIN,ARI,1st_long,2x2,POST,VERTICAL,51,DEEP,TRADITIONAL,ZONE_COVERAGE,COVER_2_ZONE,None,0,53434
33966,2023122410,551,53435,Targeted Receiver,0.050912,0.795257,0.744346,4,C,50,2023,16,MIA,DAL,(6:32) (Shotgun) T.Tagovailoa pass deep right ...,1,06:32,WR,52413.0,28,LONG,MIA,DAL,3rd_long,3x1,GO,VERTICAL,43,DEEP,TRADITIONAL,ZONE_COVERAGE,COVER_4_ZONE,None,0,53435
1299,2023092402,1078,43584,Targeted Receiver,0.050099,0.773502,0.723403,1,I,0,2023,3,DET,ATL,(11:32) J.Goff pass incomplete deep left to K....,2,11:32,WR,43290.0,24,LONG,DET,ATL,1st_long,2x1,POST,VERTICAL,44,DEEP,TRADITIONAL,ZONE_COVERAGE,COVER_2_ZONE,None,0,43584


In [13]:
# 2023120304	1700	
player_impacts[(player_impacts['game_id'] ==2023120301) &
               (player_impacts['play_id'] == 2032) 
              ]

,game_id,play_id,nfl_id,player_role,baseline_prob,real_prob,delta,fold,pass_result,yards_gained,season,week,home_team_abbr,visitor_team_abbr,play_description,quarter,game_clock,player_position,passer_nfl_id,num_output_frames,num_output_frames_bin,possession_team,defensive_team,down_and_distance,receiver_alignment,route_of_targeted_receiver,route_type_of_targeted_receiver,pass_length,pass_length_bin,dropback_type,team_coverage_man_zone,team_coverage_type,defender_type,potential_contact,targeted_receiver_id
32282,2023120301,2032,38607,Defensive Coverage,0.335475,0.335475,0.000000,4,C,24,2023,13,NO,DET,(2:00) (Shotgun) J.Goff pass deep right to A.S...,2,02:00,MLB,43290.0,12,MEDIUM,DET,NO,3rd_medium,2x2,POST,VERTICAL,18,INTERMEDIATE,TRADITIONAL,ZONE_COVERAGE,COVER_3_ZONE,Converging,0,53541
32283,2023120301,2032,40017,Defensive Coverage,0.335475,0.335475,0.000000,4,C,24,2023,13,NO,DET,(2:00) (Shotgun) J.Goff pass deep right to A.S...,2,02:00,FS,43290.0,12,MEDIUM,DET,NO,3rd_medium,2x2,POST,VERTICAL,18,INTERMEDIATE,TRADITIONAL,ZONE_COVERAGE,COVER_3_ZONE,Converging,0,53541
32284,2023120301,2032,46168,Defensive Coverage,0.335475,0.335475,0.000000,4,C,24,2023,13,NO,DET,(2:00) (Shotgun) J.Goff pass deep right to A.S...,2,02:00,CB,43290.0,12,MEDIUM,DET,NO,3rd_medium,2x2,POST,VERTICAL,18,INTERMEDIATE,TRADITIONAL,ZONE_COVERAGE,COVER_3_ZONE,Converging,0,53541
32285,2023120301,2032,47837,Defensive Coverage,0.335475,0.335475,0.000000,4,C,24,2023,13,NO,DET,(2:00) (Shotgun) J.Goff pass deep right to A.S...,2,02:00,FS,43290.0,12,MEDIUM,DET,NO,3rd_medium,2x2,POST,VERTICAL,18,INTERMEDIATE,TRADITIONAL,ZONE_COVERAGE,COVER_3_ZONE,Converging,0,53541
32286,2023120301,2032,53505,Defensive Coverage,0.631174,0.335475,-0.295699,4,C,24,2023,13,NO,DET,(2:00) (Shotgun) J.Goff pass deep right to A.S...,2,02:00,CB,43290.0,12,MEDIUM,DET,NO,3rd_medium,2x2,POST,VERTICAL,18,INTERMEDIATE,TRADITIONAL,ZONE_COVERAGE,COVER_3_ZONE,Trailing,1,53541
32287,2023120301,2032,53541,Targeted Receiver,0.495267,0.335475,-0.159792,4,C,24,2023,13,NO,DET,(2:00) (Shotgun) J.Goff pass deep right to A.S...,2,02:00,WR,43290.0,12,MEDIUM,DET,NO,3rd_medium,2x2,POST,VERTICAL,18,INTERMEDIATE,TRADITIONAL,ZONE_COVERAGE,COVER_3_ZONE,None,0,53541
32288,2023120301,2032,56011,Defensive Coverage,0.335475,0.335475,0.000000,4,C,24,2023,13,NO,DET,(2:00) (Shotgun) J.Goff pass deep right to A.S...,2,02:00,FS,43290.0,12,MEDIUM,DET,NO,3rd_medium,2x2,POST,VERTICAL,18,INTERMEDIATE,TRADITIONAL,ZONE_COVERAGE,COVER_3_ZONE,Converging,0,53541


In [14]:
local_submission = pd.read_csv('local_submission.csv')
local_submission[(local_submission['game_id'] == 2023120304) &
                    (local_submission['play_id'] == 1700)&
                    (local_submission['nfl_id'].isin([41233, 45004]))]
# train_input[(train_input['game_id'] == 2023120304)&
#                     (train_input['play_id'] == 1700)&
#                     (train_input['nfl_id'].isin([41233, 45004]))]

,game_id,play_id,nfl_id,frame_id,pred_x,pred_y,actual_x,actual_y,error
380075,2023120304,1700,45004,1,66.183486,25.727335,66.20,25.68,0.050134
380076,2023120304,1700,45004,2,65.686832,25.178041,65.71,25.14,0.044541
380077,2023120304,1700,45004,3,65.204105,24.594663,65.23,24.55,0.051627
380078,2023120304,1700,45004,4,64.738901,23.990867,64.78,23.87,0.127664
380079,2023120304,1700,45004,5,64.277967,23.365867,64.31,23.16,0.208344
380080,2023120304,1700,45004,6,63.836541,22.715394,63.86,22.46,0.256469
380081,2023120304,1700,45004,7,63.410735,22.049510,63.42,21.74,0.309648
380082,2023120304,1700,45004,8,62.986745,21.375515,62.96,21.01,0.366492
380083,2023120304,1700,45004,9,62.578133,20.685969,62.52,20.29,0.400214
380084,2023120304,1700,45004,10,62.172097,19.984677,62.06,19.57,0.429561


In [15]:
catch_probabilities = pd.read_csv('./outputs/catch_probabilities_lgb.csv')
catch_probabilities = catch_probabilities.rename(columns={
  'pred_catch_prob_by_proj_traj': 'baseline_prob',
  'pred_catch_prob_by_real_traj': 'real_prob',
})
catch_probabilities['delta'] = catch_probabilities['real_prob'] - catch_probabilities['baseline_prob']

catch_probabilities
print(catch_probabilities.shape)
catch_probabilities = catch_probabilities.merge(supplementary_data[['game_id','play_id','pass_result', 'yards_gained', 'season','week','home_team_abbr','visitor_team_abbr','play_description', 'quarter','game_clock']],
        on=['game_id','play_id'], how='left')
print(catch_probabilities.shape)
catch_probabilities = catch_probabilities.merge(train_input_pre[['game_id','play_id','passer_nfl_id','num_output_frames', 'num_output_frames_bin']].drop_duplicates(),
        on=['game_id','play_id'], how='left')
print(catch_probabilities.shape)
catch_probabilities = catch_probabilities.merge(suppl_data_pre,
        on=['game_id','play_id'], how='left')
print(catch_probabilities.shape)
catch_probabilities.sort_values('delta', ascending = False).head(3)

(13132, 5)
(13132, 14)
(13132, 17)
(13132, 28)


,game_id,play_id,baseline_prob,real_prob,delta,pass_result,yards_gained,season,week,home_team_abbr,visitor_team_abbr,play_description,quarter,game_clock,passer_nfl_id,num_output_frames,num_output_frames_bin,possession_team,defensive_team,down_and_distance,receiver_alignment,route_of_targeted_receiver,route_type_of_targeted_receiver,pass_length,pass_length_bin,dropback_type,team_coverage_man_zone,team_coverage_type
10937,2023100807,2356,0.057876,0.853926,0.796051,C,63,2023,5,ARI,CIN,(13:47) (Shotgun) J.Burrow pass deep right to ...,3,13:47,52409.0,29,LONG,CIN,ARI,1st_long,2x2,POST,VERTICAL,51,DEEP,TRADITIONAL,ZONE_COVERAGE,COVER_2_ZONE
7173,2023120307,2246,0.126524,0.860678,0.734154,C,39,2023,13,HOU,DEN,(11:44) (Shotgun) C.Stroud pass deep middle to...,3,11:44,55866.0,14,MEDIUM,HOU,DEN,3rd_long,4x1,POST,VERTICAL,23,DEEP,TRADITIONAL,MAN_COVERAGE,COVER_2_MAN
10107,2023122410,551,0.069271,0.795257,0.725986,C,50,2023,16,MIA,DAL,(6:32) (Shotgun) T.Tagovailoa pass deep right ...,1,06:32,52413.0,28,LONG,MIA,DAL,3rd_long,3x1,GO,VERTICAL,43,DEEP,TRADITIONAL,ZONE_COVERAGE,COVER_4_ZONE


In [16]:
# 2023120304	1700	
catch_probabilities[(catch_probabilities['game_id'] == 2023120304)&
                    (catch_probabilities['play_id'] == 1700)]

,game_id,play_id,baseline_prob,real_prob,delta,pass_result,yards_gained,season,week,home_team_abbr,visitor_team_abbr,play_description,quarter,game_clock,passer_nfl_id,num_output_frames,num_output_frames_bin,possession_team,defensive_team,down_and_distance,receiver_alignment,route_of_targeted_receiver,route_type_of_targeted_receiver,pass_length,pass_length_bin,dropback_type,team_coverage_man_zone,team_coverage_type
9649,2023120304,1700,0.133238,0.438982,0.305743,IN,0,2023,13,TB,CAR,(2:31) (Shotgun) B.Mayfield pass deep left int...,2,02:31,46070.0,27,LONG,TB,CAR,3rd_short,3x1,GO,VERTICAL,28,DEEP,TRADITIONAL,ZONE_COVERAGE,COVER_3_ZONE


In [17]:
min_plays = 30
player_stats = (player_impacts[player_impacts['player_role'] == 'Targeted Receiver']
    .groupby('nfl_id').agg(
      mean_delta=('delta', 'mean'),
      std_delta=('delta', 'std'),
      n_plays=('delta', 'count')
).query(f'n_plays >= {min_plays}')
)

In [18]:
# Variance between players (signal)
var_between = player_stats['mean_delta'].var()

# Average variance within players (noise)
var_within = (player_stats['std_delta'] ** 2).mean()

# ICC
icc = var_between / (var_between + var_within)
print(f"ICC for WR delta: {icc:.3f}")

ICC for WR delta: 0.022


In [19]:
from typing import Union, List, Dict, Callable

def apply_filter_dict(
    df: pd.DataFrame, 
    filter_dict: Dict[str, Union[str, List, Callable]]
) -> pd.DataFrame:
    filtered_df = df.copy()
    
    for col, condition in filter_dict.items():
        if callable(condition):
            # Lambda/function filter
            filtered_df = filtered_df[filtered_df[col].apply(condition)]
        elif isinstance(condition, list):
            # List of values (OR condition)
            filtered_df = filtered_df[filtered_df[col].isin(condition)]
        else:
            # Single value equality
            filtered_df = filtered_df[filtered_df[col] == condition]
    
    return filtered_df


def assess_icc_by_cut(
    df: pd.DataFrame, 
    min_plays: int,
    filter_dict: Dict[str, Union[str, List, Callable]] = None,
    groupby_cols: Union[str, List[str]] = 'nfl_id',
    label: str = 'default'
) -> Dict[str, float]:
    """
    Assess ICC with flexible filtering and grouping.
    
    Args:
        df: Input dataframe (player_impacts)
        min_plays: Minimum plays threshold
        filter_dict: Dictionary of {column: value/list/callable}
            Examples:
                {'player_role': 'Targeted Receiver'}
                {'player_role': ['Targeted Receiver', 'Defensive Coverage']}
                {'yards_gained': lambda x: x > 10}
        groupby_cols: Column(s) to group by. Can be string or list of strings
            Examples: 'nfl_id', ['nfl_id', 'quarter'], ['game_id', 'play_id']
    
    Returns:
        Dictionary with ICC metrics and metadata
    """
    filtered_df = df.copy()
    
    if filter_dict:
        filtered_df = apply_filter_dict(filtered_df, filter_dict)
    
    if isinstance(groupby_cols, str):
        groupby_cols = [groupby_cols]
    
    player_stats = (
        filtered_df
        .groupby(groupby_cols)
        .agg(
            mean_delta=('delta', 'mean'),
            std_delta=('delta', 'std'),
            n_plays=('delta', 'count')
        )
        .query(f'n_plays >= {min_plays}')
    )
    
    # Handle case where no groups survive
    if len(player_stats) == 0:
        return
    
    var_between = player_stats['mean_delta'].var()
    var_within = (player_stats['std_delta'] ** 2).mean()
    icc = var_between / (var_between + var_within) if (var_between + var_within) > 0 else np.nan
    
    return {
        'label': label,
        'icc': icc,
        'var_between': var_between,
        'var_within': var_within,
        'n_groups': len(player_stats),
        'groupby': groupby_cols,
        'filters': filter_dict,
    }

In [20]:
supplementary_data['team_coverage_man_zone'].value_counts()

team_coverage_man_zone
ZONE_COVERAGE    12783
MAN_COVERAGE      5221
Name: count, dtype: int64

In [21]:
# ======================
# Variation By Team - Unfiltered
# ======================

print(
  assess_icc_by_cut(
      catch_probabilities,
      min_plays=20,
      # filter_dict={'player_role': ['Targeted Receiver', 'Defensive Coverage']},
      groupby_cols='defensive_team',
      label = 'Variation by Defensive Team'
  )
)

print(
  assess_icc_by_cut(
      catch_probabilities,
      min_plays=20,
      # filter_dict={'player_role': 'Defensive Coverage'},
      groupby_cols='possession_team',
      label = 'Variation by Offensive Team'
  )
)

{'label': 'Variation by Defensive Team', 'icc': np.float64(0.002175074439863745), 'var_between': np.float64(2.379445735300639e-05), 'var_within': np.float64(0.010915811524360872), 'n_groups': 32, 'groupby': ['defensive_team'], 'filters': None}
{'label': 'Variation by Offensive Team', 'icc': np.float64(0.005312275729223787), 'var_between': np.float64(5.8069841332946076e-05), 'var_within': np.float64(0.010873185291659003), 'n_groups': 32, 'groupby': ['possession_team'], 'filters': None}


In [22]:
# ======================
# Variation by Defensive Team
# ======================
print(
  assess_icc_by_cut(
      catch_probabilities,
      min_plays=20,
      # filter_dict={'player_role': ['Targeted Receiver', 'Defensive Coverage']},
      groupby_cols='defensive_team',
      label = 'Variation by Defensive Team'
  )
)

print(
  assess_icc_by_cut(
      catch_probabilities,
      min_plays=20,
      filter_dict={'down_and_distance': "3rd_long"},
      groupby_cols='defensive_team',
      label = 'Variation by Defensive Team - On 3rd and Long'
  )
)

print(
  assess_icc_by_cut(
      catch_probabilities,
      min_plays=20,
      filter_dict={'route_type_of_targeted_receiver': "VERTICAL"},
      groupby_cols='defensive_team',
      label = 'Variation by Defensive Team - On Vertical Routes'
  )
)

print(
  assess_icc_by_cut(
      catch_probabilities,
      min_plays=20,
      filter_dict={'route_type_of_targeted_receiver': "OUT_BREAK"},
      groupby_cols='defensive_team',
      label = 'Variation by Defensive Team - On Out Breaking Routes'
  )
)

print(
  assess_icc_by_cut(
      catch_probabilities,
      min_plays=20,
      filter_dict={'route_type_of_targeted_receiver': "IN_BREAK"},
      groupby_cols='defensive_team',
      label = 'Variation by Defensive Team - On In Breaking Routes'
  )
)

print(
  assess_icc_by_cut(
      catch_probabilities,
      min_plays=20,
      filter_dict={'pass_length_bin': "SHORT"},
      groupby_cols='defensive_team',
      label = 'Variation by Defensive Team - On Short Pass Lengths'
  )
)

print(
  assess_icc_by_cut(
      catch_probabilities,
      min_plays=20,
      filter_dict={'pass_length_bin': "INTERMEDIATE"},
      groupby_cols='defensive_team',
      label = 'Variation by Defensive Team - On Intermediate Pass Lengths'
  )
)

print(
  assess_icc_by_cut(
      catch_probabilities,
      min_plays=20,
      filter_dict={'pass_length_bin': "DEEP"},
      groupby_cols='defensive_team',
      label = 'Variation by Defensive Team - On Deep Pass Lengths'
  )
)


print(
  assess_icc_by_cut(
      catch_probabilities,
      min_plays=20,
      filter_dict={'num_output_frames_bin': "QUICK"},
      groupby_cols='defensive_team',
      label = 'Variation by Defensive Team - On Quick Passes'
  )
)

print(
  assess_icc_by_cut(
      catch_probabilities,
      min_plays=20,
      filter_dict={'num_output_frames_bin': "MEDIUM"},
      groupby_cols='defensive_team',
      label = 'Variation by Defensive Team - On Medium-Time Passes'
  )
)

print(
  assess_icc_by_cut(
      catch_probabilities,
      min_plays=20,
      filter_dict={'num_output_frames_bin': "LONG"},
      groupby_cols='defensive_team',
      label = 'Variation by Defensive Team - On Long-Time Passes'
  )
)

print(
  assess_icc_by_cut(
      catch_probabilities,
      min_plays=20,
      filter_dict={'team_coverage_man_zone': "ZONE_COVERAGE"},
      groupby_cols='defensive_team',
      label = 'Variation by Defensive Team - Playing Zone'
  )
)

print(
  assess_icc_by_cut(
      catch_probabilities,
      min_plays=20,
      filter_dict={'team_coverage_man_zone': "MAN_COVERAGE"},
      groupby_cols='defensive_team',
      label = 'Variation by Defensive Team - Playing Man'
  )
)



{'label': 'Variation by Defensive Team', 'icc': np.float64(0.002175074439863745), 'var_between': np.float64(2.379445735300639e-05), 'var_within': np.float64(0.010915811524360872), 'n_groups': 32, 'groupby': ['defensive_team'], 'filters': None}
{'label': 'Variation by Defensive Team - On 3rd and Long', 'icc': np.float64(0.027951195252041292), 'var_between': np.float64(0.0003544406659684245), 'var_within': np.float64(0.01232625734255554), 'n_groups': 32, 'groupby': ['defensive_team'], 'filters': {'down_and_distance': '3rd_long'}}
{'label': 'Variation by Defensive Team - On Vertical Routes', 'icc': np.float64(0.012528463144996808), 'var_between': np.float64(0.0003174058747694098), 'var_within': np.float64(0.025017375502319493), 'n_groups': 32, 'groupby': ['defensive_team'], 'filters': {'route_type_of_targeted_receiver': 'VERTICAL'}}
{'label': 'Variation by Defensive Team - On Out Breaking Routes', 'icc': np.float64(0.006750327023712798), 'var_between': np.float64(4.1935989089429674e-05), 

In [23]:
# ======================
# Variation by Offensive Team
# ======================
print(
  assess_icc_by_cut(
      catch_probabilities,
      min_plays=20,
      # filter_dict={'player_role': ['Targeted Receiver', 'Defensive Coverage']},
      groupby_cols='possession_team',
      label = 'Variation by Offensive Team'
  )
)

print(
  assess_icc_by_cut(
      catch_probabilities,
      min_plays=20,
      filter_dict={'down_and_distance': "3rd_long"},
      groupby_cols='possession_team',
      label = 'Variation by Offensive Team - On 3rd and Long'
  )
)

print(
  assess_icc_by_cut(
      catch_probabilities,
      min_plays=20,
      filter_dict={'route_type_of_targeted_receiver': "VERTICAL"},
      groupby_cols='possession_team',
      label = 'Variation by Offensive Team - On Vertical Routes'
  )
)

print(
  assess_icc_by_cut(
      catch_probabilities,
      min_plays=20,
      filter_dict={'route_type_of_targeted_receiver': "OUT_BREAK"},
      groupby_cols='possession_team',
      label = 'Variation by Offensive Team - On Out Breaking Routes'
  )
)

print(
  assess_icc_by_cut(
      catch_probabilities,
      min_plays=20,
      filter_dict={'route_type_of_targeted_receiver': "IN_BREAK"},
      groupby_cols='possession_team',
      label = 'Variation by Offensive Team - On In Breaking Routes'
  )
)

print(
  assess_icc_by_cut(
      catch_probabilities,
      min_plays=20,
      filter_dict={'pass_length_bin': "SHORT"},
      groupby_cols='possession_team',
      label = 'Variation by Offensive Team - On Short Pass Lengths'
  )
)

print(
  assess_icc_by_cut(
      catch_probabilities,
      min_plays=20,
      filter_dict={'pass_length_bin': "INTERMEDIATE"},
      groupby_cols='possession_team',
      label = 'Variation by Offensive Team - On Intermediate Pass Lengths'
  )
)

print(
  assess_icc_by_cut(
      catch_probabilities,
      min_plays=20,
      filter_dict={'pass_length_bin': "DEEP"},
      groupby_cols='possession_team',
      label = 'Variation by Offensive Team - On Deep Pass Lengths'
  )
)


print(
  assess_icc_by_cut(
      catch_probabilities,
      min_plays=20,
      filter_dict={'num_output_frames_bin': "QUICK"},
      groupby_cols='possession_team',
      label = 'Variation by Offensive Team - On Quick Passes'
  )
)

print(
  assess_icc_by_cut(
      catch_probabilities,
      min_plays=20,
      filter_dict={'num_output_frames_bin': "MEDIUM"},
      groupby_cols='possession_team',
      label = 'Variation by Offensive Team - On Medium-Time Passes'
  )
)

print(
  assess_icc_by_cut(
      catch_probabilities,
      min_plays=20,
      filter_dict={'num_output_frames_bin': "LONG"},
      groupby_cols='possession_team',
      label = 'Variation by Offensive Team - On Long-Time Passes'
  )
)


print(
  assess_icc_by_cut(
      catch_probabilities,
      min_plays=20,
      filter_dict={'team_coverage_man_zone': "MAN_COVERAGE"},
      groupby_cols='possession_team',
      label = 'Variation by Offensive Team - Playing Man'
  )
)
print(
  assess_icc_by_cut(
      catch_probabilities,
      min_plays=20,
      filter_dict={'team_coverage_man_zone': "ZONE_COVERAGE"},
      groupby_cols='possession_team',
      label = 'Variation by Offensive Team - Playing Zone'
  )
)

{'label': 'Variation by Offensive Team', 'icc': np.float64(0.005312275729223787), 'var_between': np.float64(5.8069841332946076e-05), 'var_within': np.float64(0.010873185291659003), 'n_groups': 32, 'groupby': ['possession_team'], 'filters': None}
{'label': 'Variation by Offensive Team - On 3rd and Long', 'icc': np.float64(0.029680071853328688), 'var_between': np.float64(0.00037868050745296626), 'var_within': np.float64(0.012380065809749645), 'n_groups': 32, 'groupby': ['possession_team'], 'filters': {'down_and_distance': '3rd_long'}}
{'label': 'Variation by Offensive Team - On Vertical Routes', 'icc': np.float64(0.013818714484172), 'var_between': np.float64(0.0003486254188845475), 'var_within': np.float64(0.0248798731714774), 'n_groups': 32, 'groupby': ['possession_team'], 'filters': {'route_type_of_targeted_receiver': 'VERTICAL'}}
{'label': 'Variation by Offensive Team - On Out Breaking Routes', 'icc': np.float64(0.011944365458559827), 'var_between': np.float64(7.486729332107184e-05), 

In [24]:
# ======================
# Variation by Passer
# ======================
print(
  assess_icc_by_cut(
      catch_probabilities,
      min_plays=20,
      # filter_dict={'player_role': ['Targeted Receiver', 'Defensive Coverage']},
      groupby_cols='passer_nfl_id',
      label = 'Variation by Passer'
  )
)

print(
  assess_icc_by_cut(
      catch_probabilities,
      min_plays=20,
      filter_dict={'down_and_distance': "3rd_long"},
      groupby_cols='passer_nfl_id',
      label = 'Variation by Passer - On 3rd and Long'
  )
)

print(
  assess_icc_by_cut(
      catch_probabilities,
      min_plays=20,
      filter_dict={'route_type_of_targeted_receiver': "VERTICAL"},
      groupby_cols='passer_nfl_id',
      label = 'Variation by Passer - On Vertical Routes'
  )
)

print(
  assess_icc_by_cut(
      catch_probabilities,
      min_plays=20,
      filter_dict={'route_type_of_targeted_receiver': "OUT_BREAK"},
      groupby_cols='passer_nfl_id',
      label = 'Variation by Passer - On Out Breaking Routes'
  )
)

print(
  assess_icc_by_cut(
      catch_probabilities,
      min_plays=20,
      filter_dict={'route_type_of_targeted_receiver': "IN_BREAK"},
      groupby_cols='passer_nfl_id',
      label = 'Variation by Passer - On In Breaking Routes'
  )
)

print(
  assess_icc_by_cut(
      catch_probabilities,
      min_plays=20,
      filter_dict={'pass_length_bin': "SHORT"},
      groupby_cols='passer_nfl_id',
      label = 'Variation by Passer - On Short Pass Lengths'
  )
)

print(
  assess_icc_by_cut(
      catch_probabilities,
      min_plays=20,
      filter_dict={'pass_length_bin': "INTERMEDIATE"},
      groupby_cols='passer_nfl_id',
      label = 'Variation by Passer - On Intermediate Pass Lengths'
  )
)

print(
  assess_icc_by_cut(
      catch_probabilities,
      min_plays=20,
      filter_dict={'pass_length_bin': "DEEP"},
      groupby_cols='passer_nfl_id',
      label = 'Variation by Passer - On Deep Pass Lengths'
  )
)


print(
  assess_icc_by_cut(
      catch_probabilities,
      min_plays=20,
      filter_dict={'num_output_frames_bin': "QUICK"},
      groupby_cols='passer_nfl_id',
      label = 'Variation by Passer - On Quick Passes'
  )
)

print(
  assess_icc_by_cut(
      catch_probabilities,
      min_plays=20,
      filter_dict={'num_output_frames_bin': "MEDIUM"},
      groupby_cols='passer_nfl_id',
      label = 'Variation by Passer - On Medium-Time Passes'
  )
)

print(
  assess_icc_by_cut(
      catch_probabilities,
      min_plays=20,
      filter_dict={'num_output_frames_bin': "LONG"},
      groupby_cols='passer_nfl_id',
      label = 'Variation by Passer - On Long-Time Passes'
  )
)


print(
  assess_icc_by_cut(
      catch_probabilities,
      min_plays=20,
      filter_dict={'team_coverage_man_zone': "MAN_COVERAGE"},
      groupby_cols='passer_nfl_id',
      label = 'Variation by Passer - Playing Man'
  )
)
print(
  assess_icc_by_cut(
      catch_probabilities,
      min_plays=20,
      filter_dict={'team_coverage_man_zone': "ZONE_COVERAGE"},
      groupby_cols='passer_nfl_id',
      label = 'Variation by Passer - Playing Zone'
  )
)

{'label': 'Variation by Passer', 'icc': np.float64(0.02030172971161014), 'var_between': np.float64(0.00022067746264001485), 'var_within': np.float64(0.010649207309484302), 'n_groups': 65, 'groupby': ['passer_nfl_id'], 'filters': None}
{'label': 'Variation by Passer - On 3rd and Long', 'icc': np.float64(0.03950233480541827), 'var_between': np.float64(0.0004996098737699745), 'var_within': np.float64(0.012147993773735094), 'n_groups': 34, 'groupby': ['passer_nfl_id'], 'filters': {'down_and_distance': '3rd_long'}}
{'label': 'Variation by Passer - On Vertical Routes', 'icc': np.float64(0.02034164653472839), 'var_between': np.float64(0.0005165716136141322), 'var_within': np.float64(0.02487820715870464), 'n_groups': 42, 'groupby': ['passer_nfl_id'], 'filters': {'route_type_of_targeted_receiver': 'VERTICAL'}}
{'label': 'Variation by Passer - On Out Breaking Routes', 'icc': np.float64(0.01937267857956769), 'var_between': np.float64(0.00012003312150974792), 'var_within': np.float64(0.00607596713

In [25]:
# ======================
# Variation by Receiver (All Roles)
# ======================

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=15,
      filter_dict={'player_role': 'Targeted Receiver'},
      groupby_cols='nfl_id',
      label = 'Variation by Receiver ID - All'
  )
)

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_role": 'Targeted Receiver',
        'route_type_of_targeted_receiver': "VERTICAL"},
      groupby_cols='nfl_id',
      label = 'Variation by Receiver ID - On Vertical Routes'
  )
)

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_role": 'Targeted Receiver',
        'route_type_of_targeted_receiver': "OUT_BREAK"},
      groupby_cols='nfl_id',
      label = 'Variation by Receiver ID - On Out Breaking Routes'
  )
)

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_role": 'Targeted Receiver',
        'route_type_of_targeted_receiver': "IN_BREAK"},
      groupby_cols='nfl_id',
      label = 'Variation by Receiver ID - On In Breaking Routes'
  )
)

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_role": 'Targeted Receiver',
        'pass_length_bin': "SHORT"},
      groupby_cols='nfl_id',
      label = 'Variation by Receiver ID - On Short Pass Lengths'
  )
)

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_role": 'Targeted Receiver',
        'pass_length_bin': "INTERMEDIATE"},
      groupby_cols='nfl_id',
      label = 'Variation by Receiver ID - On Intermediate Pass Lengths'
  )
)

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_role": 'Targeted Receiver',
        'pass_length_bin': "DEEP"},
      groupby_cols='nfl_id',
      label = 'Variation by Receiver ID - On Deep Pass Lengths'
  )
)


print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_role": 'Targeted Receiver',
        'num_output_frames_bin': "QUICK"},
      groupby_cols='nfl_id',
      label = 'Variation by Receiver ID - On Quick Passes'
  )
)

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_role": 'Targeted Receiver',
        'num_output_frames_bin': "MEDIUM"},
      groupby_cols='nfl_id',
      label = 'Variation by Receiver ID - On Medium-Time Passes'
  )
)

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_role": 'Targeted Receiver',
        'num_output_frames_bin': "LONG"},
      groupby_cols='nfl_id',
      label = 'Variation by Receiver ID - On Long-Time Passes'
  )
)


print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_role": 'Targeted Receiver',
        'team_coverage_man_zone': "MAN_COVERAGE"},
      groupby_cols='nfl_id',
      label = 'Variation by Receiver ID - Playing Man'
  )
)
print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_role": 'Targeted Receiver',
        'team_coverage_man_zone': "ZONE_COVERAGE"},
      groupby_cols='nfl_id',
      label = 'Variation by Receiver ID - Playing Zone'
  )
)


{'label': 'Variation by Receiver ID - All', 'icc': np.float64(0.03603071819180606), 'var_between': np.float64(0.00033815201054562707), 'var_within': np.float64(0.009046951243447463), 'n_groups': 251, 'groupby': ['nfl_id'], 'filters': {'player_role': 'Targeted Receiver'}}
{'label': 'Variation by Receiver ID - On Vertical Routes', 'icc': np.float64(0.037837703293454074), 'var_between': np.float64(0.0008581516601473577), 'var_within': np.float64(0.021821651431807698), 'n_groups': 40, 'groupby': ['nfl_id'], 'filters': {'player_role': 'Targeted Receiver', 'route_type_of_targeted_receiver': 'VERTICAL'}}
{'label': 'Variation by Receiver ID - On Out Breaking Routes', 'icc': np.float64(0.06915037580082406), 'var_between': np.float64(0.0004033932967269859), 'var_within': np.float64(0.005430172928406655), 'n_groups': 62, 'groupby': ['nfl_id'], 'filters': {'player_role': 'Targeted Receiver', 'route_type_of_targeted_receiver': 'OUT_BREAK'}}
{'label': 'Variation by Receiver ID - On In Breaking Route

In [26]:
# ======================
# Variation by Defender (All Roles)
# ======================

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={'player_role': 'Defensive Coverage'},
      groupby_cols='nfl_id',
      label = 'Variation by Defender ID - All'
  )
)

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_role": 'Defensive Coverage',
        'route_type_of_targeted_receiver': "VERTICAL"},
      groupby_cols='nfl_id',
      label = 'Variation by Defender ID - On Vertical Routes'
  )
)

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_role": 'Defensive Coverage',
        'route_type_of_targeted_receiver': "OUT_BREAK"},
      groupby_cols='nfl_id',
      label = 'Variation by Defender ID - On Out Breaking Routes'
  )
)

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_role": 'Defensive Coverage',
        'route_type_of_targeted_receiver': "IN_BREAK"},
      groupby_cols='nfl_id',
      label = 'Variation by Defender ID - On In Breaking Routes'
  )
)

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_role": 'Defensive Coverage',
        'pass_length_bin': "SHORT"},
      groupby_cols='nfl_id',
      label = 'Variation by Defender ID - On Short Pass Lengths'
  )
)

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_role": 'Defensive Coverage',
        'pass_length_bin': "INTERMEDIATE"},
      groupby_cols='nfl_id',
      label = 'Variation by Defender ID - On Intermediate Pass Lengths'
  )
)

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_role": 'Defensive Coverage',
        'pass_length_bin': "DEEP"},
      groupby_cols='nfl_id',
      label = 'Variation by Defender ID - On Deep Pass Lengths'
  )
)


print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_role": 'Defensive Coverage',
        'num_output_frames_bin': "QUICK"},
      groupby_cols='nfl_id',
      label = 'Variation by Defender ID - On Quick Passes'
  )
)

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_role": 'Defensive Coverage',
        'num_output_frames_bin': "MEDIUM"},
      groupby_cols='nfl_id',
      label = 'Variation by Defender ID - On Medium-Time Passes'
  )
)

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_role": 'Defensive Coverage',
        'num_output_frames_bin': "LONG"},
      groupby_cols='nfl_id',
      label = 'Variation by Defender ID - On Long-Time Passes'
  )
)


print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_role": 'Defensive Coverage',
        'team_coverage_man_zone': "MAN_COVERAGE"},
      groupby_cols='nfl_id',
      label = 'Variation by Defender ID - Playing Man'
  )
)
print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_role": 'Defensive Coverage',
        'team_coverage_man_zone': "ZONE_COVERAGE"},
      groupby_cols='nfl_id',
      label = 'Variation by Defender ID - Playing Zone'
  )
)

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_role": 'Defensive Coverage',
        # 'team_coverage_man_zone': "ZONE_COVERAGE",
        "defender_type": "Converging"},
      groupby_cols='nfl_id',
      label = 'Variation by Defender ID - When Converging'
  )
)

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_role": 'Defensive Coverage',
        # 'team_coverage_man_zone': "ZONE_COVERAGE",
        "defender_type": "Converging",
        'pass_length_bin': ["INTERMEDIATE", "DEEP"]},
      groupby_cols='nfl_id',
      label = 'Variation by Defender ID - When Converging Deep'
  )
)


print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_role": 'Defensive Coverage',
        # 'team_coverage_man_zone': "ZONE_COVERAGE",
        "defender_type": "Converging",
        # 'pass_length_bin': ["INTERMEDIATE", "DEEP"],
        'route_type_of_targeted_receiver': "VERTICAL"
        },
      groupby_cols='nfl_id',
      label = 'Variation by Defender ID - When Converging Verts'
  )
)


print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_role": 'Defensive Coverage',
        # 'team_coverage_man_zone': "ZONE_COVERAGE",
        "defender_type": "Converging",
        'pass_length_bin': ["INTERMEDIATE", "DEEP"],
        'route_type_of_targeted_receiver': "VERTICAL"
        },
      groupby_cols='nfl_id',
      label = 'Variation by Defender ID - When Converging Deep Verts'
  )
)

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=15,
      filter_dict={
        "player_role": 'Defensive Coverage',
        # 'team_coverage_man_zone': "ZONE_COVERAGE",
        "defender_type": "Converging",
        # 'pass_length_bin': ["INTERMEDIATE", "DEEP"],
        # 'route_type_of_targeted_receiver': "VERTICAL",
        'team_coverage_man_zone': "MAN_COVERAGE"

        },
      groupby_cols='nfl_id',
      label = 'Variation by Defender ID - When Converging in Man'
  )
)


print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_role": 'Defensive Coverage',
        # 'team_coverage_man_zone': "ZONE_COVERAGE",
        "defender_type": "Trailing"},
      groupby_cols='nfl_id',
      label = 'Variation by Defender ID - When Trailing'
  )
)

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_role": 'Defensive Coverage',
        # 'team_coverage_man_zone': "ZONE_COVERAGE",
        "potential_contact": 1},
      groupby_cols='nfl_id',
      label = 'Variation by Defender ID - Potential Contact'
  )
)


print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_role": 'Defensive Coverage',
        # "defender_type": 'Trailing',
        # 'team_coverage_man_zone': "ZONE_COVERAGE"},
        "potential_contact": 1},
      groupby_cols='targeted_receiver_id',
      label = 'Variation by Targeted Receiver ID - Potential Contact'
  )
)

{'label': 'Variation by Defender ID - All', 'icc': np.float64(0.017528957984139625), 'var_between': np.float64(2.9092587711187444e-05), 'var_within': np.float64(0.0016305946417014626), 'n_groups': 376, 'groupby': ['nfl_id'], 'filters': {'player_role': 'Defensive Coverage'}}
{'label': 'Variation by Defender ID - On Vertical Routes', 'icc': np.float64(0.03592997408708782), 'var_between': np.float64(9.935647028739156e-05), 'var_within': np.float64(0.002665924407638356), 'n_groups': 200, 'groupby': ['nfl_id'], 'filters': {'player_role': 'Defensive Coverage', 'route_type_of_targeted_receiver': 'VERTICAL'}}
{'label': 'Variation by Defender ID - On Out Breaking Routes', 'icc': np.float64(0.042816268148771416), 'var_between': np.float64(7.006495015256184e-05), 'var_within': np.float64(0.0015663446012149434), 'n_groups': 123, 'groupby': ['nfl_id'], 'filters': {'player_role': 'Defensive Coverage', 'route_type_of_targeted_receiver': 'OUT_BREAK'}}
{'label': 'Variation by Defender ID - On In Breaki

In [27]:
# ======================
# Variation by Safety (All Roles)
# ======================

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=15,
      filter_dict={'player_position': ['SS', 'FS', 'S']},
      groupby_cols='nfl_id',
      label = 'Variation by Safety ID - All'
  )
)

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_position": ['SS', 'FS', 'S'],
        'route_type_of_targeted_receiver': "VERTICAL"},
      groupby_cols='nfl_id',
      label = 'Variation by Safety ID - On Vertical Routes'
  )
)

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_position": ['SS', 'FS', 'S'],
        'route_type_of_targeted_receiver': "OUT_BREAK"},
      groupby_cols='nfl_id',
      label = 'Variation by Safety ID - On Out Breaking Routes'
  )
)

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_position": ['SS', 'FS', 'S'],
        'route_type_of_targeted_receiver': "IN_BREAK"},
      groupby_cols='nfl_id',
      label = 'Variation by Safety ID - On In Breaking Routes'
  )
)

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_position": ['SS', 'FS', 'S'],
        'pass_length_bin': "SHORT"},
      groupby_cols='nfl_id',
      label = 'Variation by Safety ID - On Short Pass Lengths'
  )
)

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_position": ['SS', 'FS', 'S'],
        'pass_length_bin': "INTERMEDIATE"},
      groupby_cols='nfl_id',
      label = 'Variation by Safety ID - On Intermediate Pass Lengths'
  )
)

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_position": ['SS', 'FS', 'S'],
        'pass_length_bin': "DEEP"},
      groupby_cols='nfl_id',
      label = 'Variation by Safety ID - On Deep Pass Lengths'
  )
)


print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_position": ['SS', 'FS', 'S'],
        'num_output_frames_bin': "QUICK"},
      groupby_cols='nfl_id',
      label = 'Variation by Safety ID - On Quick Passes'
  )
)

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_position": ['SS', 'FS', 'S'],
        'num_output_frames_bin': "MEDIUM"},
      groupby_cols='nfl_id',
      label = 'Variation by Safety ID - On Medium-Time Passes'
  )
)

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_position": ['SS', 'FS', 'S'],
        'num_output_frames_bin': "LONG"},
      groupby_cols='nfl_id',
      label = 'Variation by Safety ID - On Long-Time Passes'
  )
)


print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_position": ['SS', 'FS', 'S'],
        'team_coverage_man_zone': "MAN_COVERAGE"},
      groupby_cols='nfl_id',
      label = 'Variation by Safety ID - Playing Man'
  )
)
print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_position": ['SS', 'FS', 'S'],
        'team_coverage_man_zone': "ZONE_COVERAGE"},
      groupby_cols='nfl_id',
      label = 'Variation by Safety ID - Playing Zone'
  )
)

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_position": ['SS', 'FS', 'S'],
        # 'team_coverage_man_zone': "ZONE_COVERAGE",
        "defender_type": "Converging"},
      groupby_cols='nfl_id',
      label = 'Variation by Safety ID - When Converging'
  )
)

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_position": ['SS', 'FS', 'S'],
        # 'team_coverage_man_zone': "ZONE_COVERAGE",
        "defender_type": "Converging",
        'pass_length_bin': ["INTERMEDIATE", "DEEP"]
        },
      groupby_cols='nfl_id',
      label = 'Variation by Safety ID - When Converging Deep'
  )
)

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_position": ['SS', 'FS', 'S'],
        # 'team_coverage_man_zone': "ZONE_COVERAGE",
        "defender_type": "Trailing"},
      groupby_cols='nfl_id',
      label = 'Variation by Safety ID - When Trailing'
  )
)

{'label': 'Variation by Safety ID - All', 'icc': np.float64(0.018208970957264967), 'var_between': np.float64(2.375740420459726e-05), 'var_within': np.float64(0.001280951371505685), 'n_groups': 129, 'groupby': ['nfl_id'], 'filters': {'player_position': ['SS', 'FS', 'S']}}
{'label': 'Variation by Safety ID - On Vertical Routes', 'icc': np.float64(0.026007739510297445), 'var_between': np.float64(4.006881775062982e-05), 'var_within': np.float64(0.001500580946707564), 'n_groups': 84, 'groupby': ['nfl_id'], 'filters': {'player_position': ['SS', 'FS', 'S'], 'route_type_of_targeted_receiver': 'VERTICAL'}}
{'label': 'Variation by Safety ID - On Out Breaking Routes', 'icc': np.float64(0.046470448292841846), 'var_between': np.float64(8.710886784509069e-05), 'var_within': np.float64(0.0017873914015768561), 'n_groups': 20, 'groupby': ['nfl_id'], 'filters': {'player_position': ['SS', 'FS', 'S'], 'route_type_of_targeted_receiver': 'OUT_BREAK'}}
{'label': 'Variation by Safety ID - On In Breaking Route

In [28]:
# ======================
# Variation by Cornerback (All Roles)
# ======================

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=15,
      filter_dict={'player_position': ['CB']},
      groupby_cols='nfl_id',
      label = 'Variation by Cornerback ID - All'
  )
)

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_position": ['CB'],
        'route_type_of_targeted_receiver': "VERTICAL"},
      groupby_cols='nfl_id',
      label = 'Variation by Cornerback ID - On Vertical Routes'
  )
)

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_position": ['CB'],
        'route_type_of_targeted_receiver': "OUT_BREAK"},
      groupby_cols='nfl_id',
      label = 'Variation by Cornerback ID - On Out Breaking Routes'
  )
)

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_position": ['CB'],
        'route_type_of_targeted_receiver': "IN_BREAK"},
      groupby_cols='nfl_id',
      label = 'Variation by Cornerback ID - On In Breaking Routes'
  )
)

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_position": ['CB'],
        'pass_length_bin': "SHORT"},
      groupby_cols='nfl_id',
      label = 'Variation by Cornerback ID - On Short Pass Lengths'
  )
)

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_position": ['CB'],
        'pass_length_bin': "INTERMEDIATE"},
      groupby_cols='nfl_id',
      label = 'Variation by Cornerback ID - On Intermediate Pass Lengths'
  )
)

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_position": ['CB'],
        'pass_length_bin': "DEEP"},
      groupby_cols='nfl_id',
      label = 'Variation by Cornerback ID - On Deep Pass Lengths'
  )
)


print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_position": ['CB'],
        'num_output_frames_bin': "QUICK"},
      groupby_cols='nfl_id',
      label = 'Variation by Cornerback ID - On Quick Passes'
  )
)

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_position": ['CB'],
        'num_output_frames_bin': "MEDIUM"},
      groupby_cols='nfl_id',
      label = 'Variation by Cornerback ID - On Medium-Time Passes'
  )
)

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_position": ['CB'],
        'num_output_frames_bin': "LONG"},
      groupby_cols='nfl_id',
      label = 'Variation by Cornerback ID - On Long-Time Passes'
  )
)


print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_position": ['CB'],
        'team_coverage_man_zone': "MAN_COVERAGE"},
      groupby_cols='nfl_id',
      label = 'Variation by Cornerback ID - Playing Man'
  )
)
print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_position": ['CB'],
        'team_coverage_man_zone': "ZONE_COVERAGE"},
      groupby_cols='nfl_id',
      label = 'Variation by Cornerback ID - Playing Zone'
  )
)


print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_position": ['CB'],
        # 'team_coverage_man_zone': "ZONE_COVERAGE",
        "defender_type": "Converging"},
      groupby_cols='nfl_id',
      label = 'Variation by Cornerback ID - When Converging'
  )
)

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_position": ['CB'],
        # 'team_coverage_man_zone': "ZONE_COVERAGE",
        "defender_type": "Trailing"},
      groupby_cols='nfl_id',
      label = 'Variation by Cornerback ID - When Trailing'
  )
)

{'label': 'Variation by Cornerback ID - All', 'icc': np.float64(0.017928022219617257), 'var_between': np.float64(4.961995599533225e-05), 'var_within': np.float64(0.0027181117763447216), 'n_groups': 155, 'groupby': ['nfl_id'], 'filters': {'player_position': ['CB']}}


{'label': 'Variation by Cornerback ID - On Vertical Routes', 'icc': np.float64(0.036703185604275615), 'var_between': np.float64(0.0001759393256887213), 'var_within': np.float64(0.0046176316625533955), 'n_groups': 82, 'groupby': ['nfl_id'], 'filters': {'player_position': ['CB'], 'route_type_of_targeted_receiver': 'VERTICAL'}}
{'label': 'Variation by Cornerback ID - On Out Breaking Routes', 'icc': np.float64(0.03803874940545474), 'var_between': np.float64(6.510935972180985e-05), 'var_within': np.float64(0.0016465494287364983), 'n_groups': 83, 'groupby': ['nfl_id'], 'filters': {'player_position': ['CB'], 'route_type_of_targeted_receiver': 'OUT_BREAK'}}
{'label': 'Variation by Cornerback ID - On In Breaking Routes', 'icc': np.float64(0.02670941291046386), 'var_between': np.float64(5.263628587196679e-05), 'var_within': np.float64(0.0019180654307256917), 'n_groups': 84, 'groupby': ['nfl_id'], 'filters': {'player_position': ['CB'], 'route_type_of_targeted_receiver': 'IN_BREAK'}}
{'label': 'Va

In [29]:
# ======================
# Variation by Free Safety (All Roles)
# ======================

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=15,
      filter_dict={'player_position': ['FS']},
      groupby_cols='nfl_id',
      label = 'Variation by Safety ID - All'
  )
)

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_position": ['FS'],
        'route_type_of_targeted_receiver': "VERTICAL"},
      groupby_cols='nfl_id',
      label = 'Variation by Free Safety ID - On Vertical Routes'
  )
)

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_position": ['FS'],
        'route_type_of_targeted_receiver': "OUT_BREAK"},
      groupby_cols='nfl_id',
      label = 'Variation by Free Safety ID - On Out Breaking Routes'
  )
)

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_position": ['FS'],
        'route_type_of_targeted_receiver': "IN_BREAK"},
      groupby_cols='nfl_id',
      label = 'Variation by Free Safety ID - On In Breaking Routes'
  )
)

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_position": ['FS'],
        'pass_length_bin': "SHORT"},
      groupby_cols='nfl_id',
      label = 'Variation by Free Safety ID - On Short Pass Lengths'
  )
)

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_position": ['FS'],
        'pass_length_bin': "INTERMEDIATE"},
      groupby_cols='nfl_id',
      label = 'Variation by Free Safety ID - On Intermediate Pass Lengths'
  )
)

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_position": ['FS'],
        'pass_length_bin': "DEEP"},
      groupby_cols='nfl_id',
      label = 'Variation by Free Safety ID - On Deep Pass Lengths'
  )
)


print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_position": ['FS'],
        'num_output_frames_bin': "QUICK"},
      groupby_cols='nfl_id',
      label = 'Variation by Free Safety ID - On Quick Passes'
  )
)

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_position": ['FS'],
        'num_output_frames_bin': "MEDIUM"},
      groupby_cols='nfl_id',
      label = 'Variation by Free Safety ID - On Medium-Time Passes'
  )
)

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_position": ['FS'],
        'num_output_frames_bin': "LONG"},
      groupby_cols='nfl_id',
      label = 'Variation by Free Safety ID - On Long-Time Passes'
  )
)


print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_position": ['FS'],
        'team_coverage_man_zone': "MAN_COVERAGE"},
      groupby_cols='nfl_id',
      label = 'Variation by Free Safety ID - Playing Man'
  )
)
print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_position": ['FS'],
        'team_coverage_man_zone': "ZONE_COVERAGE"},
      groupby_cols='nfl_id',
      label = 'Variation by Free Safety ID - Playing Zone'
  )
)

{'label': 'Variation by Safety ID - All', 'icc': np.float64(0.020874719046310147), 'var_between': np.float64(2.9473888276215848e-05), 'var_within': np.float64(0.0013824679065248837), 'n_groups': 73, 'groupby': ['nfl_id'], 'filters': {'player_position': ['FS']}}
{'label': 'Variation by Free Safety ID - On Vertical Routes', 'icc': np.float64(0.02684801126745065), 'var_between': np.float64(4.639405450444277e-05), 'var_within': np.float64(0.001681631684246974), 'n_groups': 49, 'groupby': ['nfl_id'], 'filters': {'player_position': ['FS'], 'route_type_of_targeted_receiver': 'VERTICAL'}}
{'label': 'Variation by Free Safety ID - On Out Breaking Routes', 'icc': np.float64(0.05610617254615678), 'var_between': np.float64(8.466749842494701e-05), 'var_within': np.float64(0.0014243910344003624), 'n_groups': 9, 'groupby': ['nfl_id'], 'filters': {'player_position': ['FS'], 'route_type_of_targeted_receiver': 'OUT_BREAK'}}
{'label': 'Variation by Free Safety ID - On In Breaking Routes', 'icc': np.float6

In [30]:
print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=20,
      filter_dict={
        "player_role": 'Defensive Coverage',
        # 'team_coverage_man_zone': "ZONE_COVERAGE",
        "defender_type": "Converging",
        'pass_length_bin': ["INTERMEDIATE", "DEEP"],
        'route_type_of_targeted_receiver': "VERTICAL"
        },
      groupby_cols='nfl_id',
      label = 'Variation by Defender ID - When Converging Deep Verts'
  )
)

print(
  assess_icc_by_cut(
      player_impacts,
      min_plays=15,
      filter_dict={
        "player_role": 'Defensive Coverage',
        # 'team_coverage_man_zone': "ZONE_COVERAGE",
        "defender_type": "Converging",
        'pass_length_bin': ["INTERMEDIATE", "DEEP"],
        # 'route_type_of_targeted_receiver': "VERTICAL",
        # 'team_coverage_man_zone': "MAN_COVERAGE"
        },
      groupby_cols='nfl_id',
      label = 'Variation by Defender ID - When Converging in Man'
  )
)


{'label': 'Variation by Defender ID - When Converging Deep Verts', 'icc': np.float64(0.039207550555493846), 'var_between': np.float64(2.7085181487025667e-05), 'var_within': np.float64(0.0006637302635811293), 'n_groups': 86, 'groupby': ['nfl_id'], 'filters': {'player_role': 'Defensive Coverage', 'defender_type': 'Converging', 'pass_length_bin': ['INTERMEDIATE', 'DEEP'], 'route_type_of_targeted_receiver': 'VERTICAL'}}
{'label': 'Variation by Defender ID - When Converging in Man', 'icc': np.float64(0.03666708695403579), 'var_between': np.float64(1.7673505225183987e-05), 'var_within': np.float64(0.0004643256578754653), 'n_groups': 252, 'groupby': ['nfl_id'], 'filters': {'player_role': 'Defensive Coverage', 'defender_type': 'Converging', 'pass_length_bin': ['INTERMEDIATE', 'DEEP']}}


In [163]:
rec_filtered = player_impacts[(player_impacts['player_role'] == 'Defensive Coverage')&
                          (player_impacts['potential_contact'] == 1)&
                          # player_impacts['defender_type'].isin(['Trailing'])
                          # player_impacts['pass_length_bin'].isin(['INTERMEDIATE', 'DEEP'])
                          (player_impacts['route_type_of_targeted_receiver'] == "VERTICAL")
                          # (player_impacts['team_coverage_man_zone'] == "MAN_COVERAGE")
                          # &(player_impacts['pass_length_bin'].isin(['DEEP']))
                          ].copy()

rec_filtered[rec_filtered['delta'] > 0.01].groupby('targeted_receiver_id').count()[['play_id']].reset_index().sort_values('play_id',ascending = False)
rec_filtered[rec_filtered['delta'] > 0.01].groupby('targeted_receiver_id').count()[['play_id']].reset_index().sort_values('play_id',ascending = False)
# rec_res = rec_filtered.groupby('targeted_receiver_id').agg(
#     mean_delta=('delta', 'mean'),
#     std_delta=('delta', 'std'),
#     n_plays=('delta', 'count')
# ).query('n_plays >= 15')

# rec_res.reset_index(inplace = True)

# rec_res.sort_values('mean_delta', ascending= False)

,targeted_receiver_id,play_id
85,54517,6
7,42347,6
49,47859,5
46,47847,5
76,53511,4
...,...,...
51,47991,1
48,47852,1
45,47839,1
43,47819,1


In [128]:
train_input[train_input['nfl_id'] == 41282].head(1)

,game_id,play_id,player_to_predict,nfl_id,frame_id,play_direction,absolute_yardline_number,player_name,player_height,player_weight,player_birth_date,player_position,player_side,player_role,x,y,s,a,dir,o,num_frames_output,ball_land_x,ball_land_y
179238,2023091009,103,True,41282,1,left,53,Davante Adams,6-1,215,1992-12-24,WR,Offense,Targeted Receiver,53.09,21.06,0.01,0.23,236.19,97.42,7,49.330002,25.59


In [123]:
rec_filtered[rec_filtered['targeted_receiver_id'] == 42489].sort_values('delta',ascending = False).head(20)

,game_id,play_id,nfl_id,player_role,baseline_prob,real_prob,delta,fold,pass_result,yards_gained,season,week,home_team_abbr,visitor_team_abbr,play_description,quarter,game_clock,player_position,passer_nfl_id,num_output_frames,num_output_frames_bin,possession_team,defensive_team,down_and_distance,receiver_alignment,route_of_targeted_receiver,route_type_of_targeted_receiver,pass_length,pass_length_bin,dropback_type,team_coverage_man_zone,team_coverage_type,defender_type,potential_contact,targeted_receiver_id
12149,2023102600,2860,47877,Defensive Coverage,0.191051,0.424988,0.233937,2,I,0,2023,8,BUF,TB,"(5:13) (No Huddle, Shotgun) J.Allen pass incom...",3,05:13,CB,46076.0,17,MEDIUM,BUF,TB,2nd_medium,2x2,GO,VERTICAL,26,DEEP,TRADITIONAL,MAN_COVERAGE,COVER_1_MAN,Trailing,1,42489
31980,2023112609,2513,43353,Defensive Coverage,0.151653,0.363021,0.211368,4,C,13,2023,12,PHI,BUF,(:21) (Shotgun) J.Allen pass short left to S.D...,2,00:21,FS,46076.0,10,QUICK,BUF,PHI,3rd_medium,3x2,CROSS,IN_BREAK,13,INTERMEDIATE,TRADITIONAL,ZONE_COVERAGE,COVER_4_ZONE,Trailing,1,42489
1738,2023100101,1878,54936,Defensive Coverage,0.477385,0.661431,0.184046,1,C,55,2023,4,BUF,MIA,(3:12) J.Allen pass short left to S.Diggs for ...,2,03:12,CB,46076.0,16,MEDIUM,BUF,MIA,2nd_medium,2x2,CORNER,VERTICAL,13,INTERMEDIATE,TRADITIONAL,MAN_COVERAGE,COVER_1_MAN,Trailing,1,42489
17999,2023091100,3987,46211,Defensive Coverage,0.195587,0.370443,0.174856,3,I,0,2023,1,NYJ,BUF,"(9:26) (No Huddle, Shotgun) J.Allen pass incom...",5,09:26,CB,46076.0,11,MEDIUM,BUF,NYJ,3rd_long,3x2,HITCH,STOP_HITCH,12,INTERMEDIATE,SCRAMBLE,MAN_COVERAGE,COVER_1_MAN,Trailing,1,42489
20094,2023100800,3466,46456,Defensive Coverage,0.303373,0.462522,0.159150,3,IN,0,2023,5,BUF,JAX,(13:16) (Shotgun) J.Allen pass deep right inte...,4,13:16,CB,46076.0,29,LONG,BUF,JAX,3rd_long,2x1,POST,VERTICAL,52,DEEP,TRADITIONAL,ZONE_COVERAGE,COVER_3_ZONE,Trailing,1,42489
26207,2024010713,1344,43299,Defensive Coverage,0.380446,0.432664,0.052218,3,C,36,2023,18,MIA,BUF,(5:56) (Shotgun) J.Allen pass deep left to S.D...,2,05:56,CB,46076.0,29,LONG,BUF,MIA,2nd_medium,2x2,GO,VERTICAL,35,DEEP,TRADITIONAL,MAN_COVERAGE,COVER_0_MAN,Trailing,1,42489
11454,2023101512,845,44830,Defensive Coverage,0.357832,0.394427,0.036594,2,C,28,2023,6,BUF,NYG,(3:19) D.Edwards reported in as eligible. J.A...,1,03:19,CB,46076.0,16,MEDIUM,BUF,NYG,1st_long,2x1,CROSS,IN_BREAK,25,DEEP,DESIGNED_ROLLOUT_LEFT,ZONE_COVERAGE,COVER_3_ZONE,Trailing,1,42489
6153,2023111909,2511,53629,Defensive Coverage,0.043065,0.041667,-0.001398,1,I,0,2023,11,BUF,NYJ,(:12) (Shotgun) J.Allen pass incomplete deep l...,2,00:12,CB,46076.0,20,MEDIUM,BUF,NYJ,2nd_short,3x1,CORNER,VERTICAL,27,DEEP,TRADITIONAL,MAN_COVERAGE,COVER_2_MAN,Trailing,1,42489
33414,2023121009,2056,52546,Defensive Coverage,0.300277,0.273799,-0.026478,4,I,0,2023,14,KC,BUF,"(:53) (No Huddle, Shotgun) J.Allen pass incomp...",2,00:53,CB,46076.0,18,MEDIUM,BUF,KC,2nd_medium,2x2,GO,VERTICAL,33,DEEP,TRADITIONAL,MAN_COVERAGE,COVER_2_MAN,Trailing,1,42489
33359,2023121009,816,52546,Defensive Coverage,0.180171,0.113704,-0.066467,4,I,0,2023,14,KC,BUF,(2:19) J.Allen pass incomplete deep left to S....,1,02:19,CB,46076.0,11,MEDIUM,BUF,KC,1st_long,2x1,HITCH,STOP_HITCH,16,INTERMEDIATE,TRADITIONAL,ZONE_COVERAGE,COVER_3_ZONE,Trailing,1,42489


In [179]:
def quick_analysis(df, filter_dict, label="Analysis"):
    """One-liner to get calibration + top players."""
    filtered = apply_filter_dict(df, filter_dict)
    
    # Calibration
    actual = (filtered['pass_result'] == 'C').mean()
    predicted = filtered['real_prob'].mean()
    
    # Top players
    top_receivers = (
        filtered.groupby('targeted_receiver_id')['delta']
        .agg(['mean', 'count'])
        .query('count >= 15')
        .sort_values('mean', ascending=False)
        .head()
    )
    
    print(f"\n{'='*60}")
    print(f"📊 {label} ({len(filtered):,} plays)")
    print(f"   Actual: {actual:.1%} | Predicted: {predicted:.1%} | Error: {abs(actual-predicted):.3f}")
    print(f"\n🎯 Top Receivers:")
    print(top_receivers)
    
    return filtered

In [203]:
# KEEP THIS, this is good for help coverage analysis
filtered = player_impacts[(player_impacts['player_role'] == 'Defensive Coverage')#&
                          # player_impacts['defender_type'].isin(['Converging'])
                          # player_impacts['pass_length_bin'].isin(['INTERMEDIATE', 'DEEP'])
                          # (player_impacts['route_type_of_targeted_receiver'] == "VERTICAL")
                          # (player_impacts['team_coverage_man_zone'] == "MAN_COVERAGE")
                          # &(player_impacts['pass_length_bin'].isin(['DEEP']))
                          ]
res = filtered.groupby('targeted_receiver_id').agg(
    mean_delta=('delta', 'mean'),
    std_delta=('delta', 'std'),
    n_plays=('delta', 'count')
).query('n_plays >= 15')

res.reset_index(inplace= True)
res.sort_values('mean_delta', ascending= False)

filtered[filtered['delta'] < -.01].groupby('nfl_id').count().reset_index().sort_values('play_id', ascending = False)

,nfl_id,game_id,play_id,player_role,baseline_prob,real_prob,delta,fold,pass_result,yards_gained,season,week,home_team_abbr,visitor_team_abbr,play_description,quarter,game_clock,player_position,passer_nfl_id,num_output_frames,num_output_frames_bin,possession_team,defensive_team,down_and_distance,receiver_alignment,route_of_targeted_receiver,route_type_of_targeted_receiver,pass_length,pass_length_bin,dropback_type,team_coverage_man_zone,team_coverage_type,defender_type,potential_contact,targeted_receiver_id
45,43351,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30
77,44878,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27
156,46757,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27
287,53503,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26
297,53565,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53,43435,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
317,54014,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
318,54466,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
384,54778,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [225]:
filtered[(filtered['defensive_team'] == 'NYJ') & 
         (filtered['possession_team'] == 'KC'
          )].sort_values('delta', ascending = False).head(20)

,game_id,play_id,nfl_id,player_role,baseline_prob,real_prob,delta,fold,pass_result,yards_gained,season,week,home_team_abbr,visitor_team_abbr,play_description,quarter,game_clock,player_position,passer_nfl_id,num_output_frames,num_output_frames_bin,possession_team,defensive_team,down_and_distance,receiver_alignment,route_of_targeted_receiver,route_type_of_targeted_receiver,pass_length,pass_length_bin,dropback_type,team_coverage_man_zone,team_coverage_type,defender_type,potential_contact,targeted_receiver_id
19704,2023100113,1796,52566,Defensive Coverage,0.480984,0.756839,0.275855,3,C,15,2023,4,NYJ,KC,(3:47) (Shotgun) P.Mahomes pass short right to...,2,03:47,CB,44822.0,18,MEDIUM,KC,NYJ,3rd_medium,2x1,WHEEL,VERTICAL,14,INTERMEDIATE,TRADITIONAL,MAN_COVERAGE,COVER_1_MAN,Trailing,1,53449
19633,2023100113,209,53583,Defensive Coverage,0.480612,0.509808,0.029196,3,I,0,2023,4,NYJ,KC,(11:43) (Shotgun) P.Mahomes pass incomplete sh...,1,11:43,CB,44822.0,11,MEDIUM,KC,NYJ,3rd_short,3x1,OUT,OUT_BREAK,5,SHORT,DESIGNED_ROLLOUT_RIGHT,MAN_COVERAGE,COVER_1_MAN,Trailing,0,54519
19656,2023100113,819,46186,Defensive Coverage,0.863927,0.879868,0.015941,3,C,34,2023,4,NYJ,KC,(3:32) (Shotgun) P.Mahomes pass deep right to ...,1,03:32,SS,44822.0,16,MEDIUM,KC,NYJ,3rd_short,3x1,CORNER,VERTICAL,16,INTERMEDIATE,TRADITIONAL,MAN_COVERAGE,COVER_1_MAN,Trailing,0,53591
19755,2023100113,3023,53583,Defensive Coverage,0.855883,0.867610,0.011727,3,C,9,2023,4,NYJ,KC,(3:14) (Shotgun) P.Mahomes pass short right to...,3,03:14,CB,44822.0,8,QUICK,KC,NYJ,1st_long,2x2,IN,IN_BREAK,3,SHORT,TRADITIONAL,ZONE_COVERAGE,COVER_4_ZONE,Converging,0,55920
19688,2023100113,1389,52476,Defensive Coverage,0.388402,0.392926,0.004523,3,IN,0,2023,4,NYJ,KC,(9:50) P.Mahomes pass deep right intended for ...,2,09:50,SS,44822.0,15,MEDIUM,KC,NYJ,1st_long,3x1,CORNER,VERTICAL,17,INTERMEDIATE,DESIGNED_ROLLOUT_RIGHT,ZONE_COVERAGE,COVER_3_ZONE,Trailing,0,53591
19762,2023100113,3137,46211,Defensive Coverage,0.910496,0.913329,0.002834,3,C,6,2023,4,NYJ,KC,(:53) (Shotgun) P.Mahomes pass short left to M...,3,00:53,CB,44822.0,12,MEDIUM,KC,NYJ,3rd_medium,3x1,OUT,OUT_BREAK,5,SHORT,TRADITIONAL,MAN_COVERAGE,COVER_1_MAN,Trailing,0,46243
19760,2023100113,3112,47881,Defensive Coverage,0.868859,0.870141,0.001282,3,C,5,2023,4,NYJ,KC,(1:29) (Shotgun) P.Mahomes pass short right to...,3,01:29,OLB,44822.0,8,QUICK,KC,NYJ,2nd_long,2x2,HITCH,STOP_HITCH,1,SHORT,TRADITIONAL,ZONE_COVERAGE,COVER_6_ZONE,Converging,0,40011
19642,2023100113,377,42485,Defensive Coverage,0.421879,0.421939,0.000060,3,I,0,2023,4,NYJ,KC,(10:35) (Shotgun) P.Mahomes pass incomplete de...,1,10:35,FS,44822.0,15,MEDIUM,KC,NYJ,1st_long,3x1,CROSS,IN_BREAK,21,DEEP,TRADITIONAL,ZONE_COVERAGE,COVER_3_ZONE,Converging,0,53591
19626,2023100113,110,41243,Defensive Coverage,0.891099,0.891099,0.000000,3,C,16,2023,4,NYJ,KC,(14:03) (Shotgun) P.Mahomes pass short left to...,1,14:03,ILB,44822.0,12,MEDIUM,KC,NYJ,2nd_long,2x2,OUT,OUT_BREAK,13,INTERMEDIATE,TRADITIONAL,ZONE_COVERAGE,COVER_4_ZONE,Trailing,0,40011
19706,2023100113,1796,54469,Defensive Coverage,0.756839,0.756839,0.000000,3,C,15,2023,4,NYJ,KC,(3:47) (Shotgun) P.Mahomes pass short right to...,2,03:47,CB,44822.0,18,MEDIUM,KC,NYJ,3rd_medium,2x1,WHEEL,VERTICAL,14,INTERMEDIATE,TRADITIONAL,MAN_COVERAGE,COVER_1_MAN,Converging,0,53449


In [240]:
test = (player_impacts[(player_impacts['player_role'] == 'Defensive Coverage') & (
  player_impacts['pass_result'] !='C'
)]
        .groupby(['game_id','play_id'])
        .agg(delta_max = ('delta', 'max'),
             delta_min = ('delta', 'min'),
             real_prob = ('real_prob', 'first'),
        )
).reset_index()

test['diff'] = test['delta_max'] - test['delta_min']
test.sort_values('diff', ascending =False).head(20)
test[(test['delta_max'] > 0.05) & (test['delta_min'] < -0.05)]


,game_id,play_id,delta_max,delta_min,real_prob,diff
35,2023091001,3514,0.087243,-0.079908,0.272339,0.167150
1141,2023101200,2306,0.161388,-0.099295,0.296194,0.260684
1212,2023101504,2802,0.054197,-0.064118,0.302399,0.118316
1324,2023101510,3721,0.161391,-0.064727,0.607122,0.226118
1538,2023102209,3793,0.053288,-0.203046,0.529321,0.256334
1778,2023102912,2954,0.050992,-0.065360,0.624707,0.116352
1794,2023103000,627,0.089547,-0.054386,0.437381,0.143933
2011,2023110600,4080,0.051871,-0.159791,0.455441,0.211662
2579,2023112604,1641,0.067001,-0.103497,0.455206,0.170498
3328,2023121708,3805,0.083126,-0.080994,0.452249,0.164119


In [244]:
# 2023102206	1919
# 2023101200	2306
# 2023102204	3510	
	# 2023100113	1796	
player_impacts[(player_impacts['game_id'] == 2023100113) & (player_impacts['play_id'] == 1796)]

,game_id,play_id,nfl_id,player_role,baseline_prob,real_prob,delta,fold,pass_result,yards_gained,season,week,home_team_abbr,visitor_team_abbr,play_description,quarter,game_clock,player_position,passer_nfl_id,num_output_frames,num_output_frames_bin,possession_team,defensive_team,down_and_distance,receiver_alignment,route_of_targeted_receiver,route_type_of_targeted_receiver,pass_length,pass_length_bin,dropback_type,team_coverage_man_zone,team_coverage_type,defender_type,potential_contact,targeted_receiver_id
19703,2023100113,1796,46186,Defensive Coverage,0.780448,0.756839,-0.023609,3,C,15,2023,4,NYJ,KC,(3:47) (Shotgun) P.Mahomes pass short right to...,2,03:47,SS,44822.0,18,MEDIUM,KC,NYJ,3rd_medium,2x1,WHEEL,VERTICAL,14,INTERMEDIATE,TRADITIONAL,MAN_COVERAGE,COVER_1_MAN,Converging,0,53449
19704,2023100113,1796,52566,Defensive Coverage,0.480984,0.756839,0.275855,3,C,15,2023,4,NYJ,KC,(3:47) (Shotgun) P.Mahomes pass short right to...,2,03:47,CB,44822.0,18,MEDIUM,KC,NYJ,3rd_medium,2x1,WHEEL,VERTICAL,14,INTERMEDIATE,TRADITIONAL,MAN_COVERAGE,COVER_1_MAN,Trailing,1,53449
19705,2023100113,1796,53449,Targeted Receiver,0.902603,0.756839,-0.145764,3,C,15,2023,4,NYJ,KC,(3:47) (Shotgun) P.Mahomes pass short right to...,2,03:47,WR,44822.0,18,MEDIUM,KC,NYJ,3rd_medium,2x1,WHEEL,VERTICAL,14,INTERMEDIATE,TRADITIONAL,MAN_COVERAGE,COVER_1_MAN,None,0,53449
19706,2023100113,1796,54469,Defensive Coverage,0.756839,0.756839,0.000000,3,C,15,2023,4,NYJ,KC,(3:47) (Shotgun) P.Mahomes pass short right to...,2,03:47,CB,44822.0,18,MEDIUM,KC,NYJ,3rd_medium,2x1,WHEEL,VERTICAL,14,INTERMEDIATE,TRADITIONAL,MAN_COVERAGE,COVER_1_MAN,Converging,0,53449


In [242]:
# 2023100108	2805
# 2023102209	3793
player_impacts[(player_impacts['game_id'] == 2023102209)&(
player_impacts['play_id'] == 3793
)]

,game_id,play_id,nfl_id,player_role,baseline_prob,real_prob,delta,fold,pass_result,yards_gained,season,week,home_team_abbr,visitor_team_abbr,play_description,quarter,game_clock,player_position,passer_nfl_id,num_output_frames,num_output_frames_bin,possession_team,defensive_team,down_and_distance,receiver_alignment,route_of_targeted_receiver,route_type_of_targeted_receiver,pass_length,pass_length_bin,dropback_type,team_coverage_man_zone,team_coverage_type,defender_type,potential_contact,targeted_receiver_id
29484,2023102209,3793,46137,Defensive Coverage,0.732367,0.529321,-0.203046,4,I,0,2023,7,KC,LAC,(3:24) (Shotgun) J.Herbert pass incomplete dee...,4,03:24,SS,52414.0,15,MEDIUM,LAC,KC,2nd_long,2x2,CORNER,VERTICAL,22,DEEP,TRADITIONAL,ZONE_COVERAGE,COVER_2_ZONE,Converging,0,53506
29485,2023102209,3793,47913,Defensive Coverage,0.529321,0.529321,0.000000,4,I,0,2023,7,KC,LAC,(3:24) (Shotgun) J.Herbert pass incomplete dee...,4,03:24,ILB,52414.0,15,MEDIUM,LAC,KC,2nd_long,2x2,CORNER,VERTICAL,22,DEEP,TRADITIONAL,ZONE_COVERAGE,COVER_2_ZONE,Trailing,0,53506
29486,2023102209,3793,53506,Targeted Receiver,0.488004,0.529321,0.041317,4,I,0,2023,7,KC,LAC,(3:24) (Shotgun) J.Herbert pass incomplete dee...,4,03:24,WR,52414.0,15,MEDIUM,LAC,KC,2nd_long,2x2,CORNER,VERTICAL,22,DEEP,TRADITIONAL,ZONE_COVERAGE,COVER_2_ZONE,None,0,53506
29487,2023102209,3793,54708,Defensive Coverage,0.476033,0.529321,0.053288,4,I,0,2023,7,KC,LAC,(3:24) (Shotgun) J.Herbert pass incomplete dee...,4,03:24,CB,52414.0,15,MEDIUM,LAC,KC,2nd_long,2x2,CORNER,VERTICAL,22,DEEP,TRADITIONAL,ZONE_COVERAGE,COVER_2_ZONE,Trailing,0,53506


In [243]:
catch_probabilities[(catch_probabilities['game_id'] == 2023102209)&
          (catch_probabilities['play_id'] == 3793)
          ]

,game_id,play_id,baseline_prob,real_prob,delta,pass_result,yards_gained,season,week,home_team_abbr,visitor_team_abbr,play_description,quarter,game_clock,passer_nfl_id,num_output_frames,num_output_frames_bin,possession_team,defensive_team,down_and_distance,receiver_alignment,route_of_targeted_receiver,route_type_of_targeted_receiver,pass_length,pass_length_bin,dropback_type,team_coverage_man_zone,team_coverage_type
8791,2023102209,3793,0.669718,0.529321,-0.140397,I,0,2023,7,KC,LAC,(3:24) (Shotgun) J.Herbert pass incomplete dee...,4,03:24,52414.0,15,MEDIUM,LAC,KC,2nd_long,2x2,CORNER,VERTICAL,22,DEEP,TRADITIONAL,ZONE_COVERAGE,COVER_2_ZONE


In [198]:
# Usage - replaces 20+ lines:
converging_def = quick_analysis(
    player_impacts,
    {'player_role': 'Defensive Coverage'},
    "Converging Defenders"
)


📊 Converging Defenders (30,900 plays)
   Actual: 63.4% | Predicted: 63.9% | Error: 0.005

🎯 Top Receivers:
                          mean  count
targeted_receiver_id                 
40078                 0.014744     30
42486                 0.013403     18
55923                 0.013280     23
42397                 0.010912     23
46189                 0.010845     30


In [208]:
train_input[train_input['nfl_id'] == 46757].head(1)

,game_id,play_id,player_to_predict,nfl_id,frame_id,play_direction,absolute_yardline_number,player_name,player_height,player_weight,player_birth_date,player_position,player_side,player_role,x,y,s,a,dir,o,num_frames_output,ball_land_x,ball_land_y
123301,2023091006,55,False,46757,1,right,35,Charvarius Ward,6-1,195,1996-05-16,CB,Defense,Defensive Coverage,42.34,15.29,0.14,0.32,327.06,324.97,9,38.950001,33.279999


In [210]:
filtered.sort_values('delta', ascending= False).head(60)

,game_id,play_id,nfl_id,player_role,baseline_prob,real_prob,delta,fold,pass_result,yards_gained,season,week,home_team_abbr,visitor_team_abbr,play_description,quarter,game_clock,player_position,passer_nfl_id,num_output_frames,num_output_frames_bin,possession_team,defensive_team,down_and_distance,receiver_alignment,route_of_targeted_receiver,route_type_of_targeted_receiver,pass_length,pass_length_bin,dropback_type,team_coverage_man_zone,team_coverage_type,defender_type,potential_contact,targeted_receiver_id
9080,2023091007,2208,52535,Defensive Coverage,0.287918,0.819573,0.531655,2,I,0,2023,1,WAS,ARI,(:19) (Shotgun) S.Howell pass incomplete short...,2,00:19,FS,54609.0,10,QUICK,WAS,ARI,1st_long,3x1,IN,IN_BREAK,11,INTERMEDIATE,TRADITIONAL,MAN_COVERAGE,COVER_1_MAN,Converging,1,54614
7509,2023121709,1807,46099,Defensive Coverage,0.204120,0.726935,0.522815,1,C,16,2023,15,CAR,ATL,(1:06) (Shotgun) B.Young pass short left to A....,2,01:06,CB,55865.0,18,MEDIUM,CAR,ATL,1st_long,2x2,GO,VERTICAL,15,INTERMEDIATE,TRADITIONAL,MAN_COVERAGE,COVER_1_MAN,Trailing,1,40488
23315,2023112302,815,55869,Defensive Coverage,0.396085,0.838639,0.442554,3,C,7,2023,12,SEA,SF,(1:36) (Shotgun) B.Purdy pass short left to J....,1,01:36,CB,54727.0,11,MEDIUM,SF,SEA,3rd_medium,2x2,OUT,OUT_BREAK,4,SHORT,TRADITIONAL,MAN_COVERAGE,COVER_1_MAN,Trailing,1,52625
6387,2023112602,295,54468,Defensive Coverage,0.313654,0.725893,0.412239,1,C,10,2023,12,HOU,JAX,(9:23) (Shotgun) T.Lawrence pass short right t...,1,09:23,CB,53430.0,14,MEDIUM,JAX,HOU,3rd_medium,2x2,HITCH,STOP_HITCH,10,SHORT,TRADITIONAL,ZONE_COVERAGE,COVER_4_ZONE,Trailing,0,44849
6598,2023121001,996,44828,Defensive Coverage,0.356796,0.716734,0.359938,1,C,37,2023,14,BAL,LA,(14:17) (Shotgun) M.Stafford pass deep right t...,2,14:17,CB,34452.0,18,MEDIUM,LA,BAL,3rd_short,3x1,GO,VERTICAL,22,DEEP,TRADITIONAL,MAN_COVERAGE,COVER_1_MAN,Trailing,1,43415
24357,2023121010,2665,45571,Defensive Coverage,0.208537,0.561158,0.352621,3,C,46,2023,14,LAC,DEN,(6:05) (Shotgun) R.Wilson pass deep right to C...,3,06:05,CB,38605.0,30,LONG,DEN,LAC,1st_long,2x2,GO,VERTICAL,46,DEEP,TRADITIONAL,ZONE_COVERAGE,COVER_6_ZONE,Trailing,1,46109
549,2023091006,3680,44962,Defensive Coverage,0.235845,0.584501,0.348656,1,I,0,2023,1,PIT,SF,(12:56) (Shotgun) B.Purdy pass incomplete shor...,4,12:56,FS,54727.0,13,MEDIUM,SF,PIT,3rd_short,3x2,OUT,OUT_BREAK,5,SHORT,TRADITIONAL,MAN_COVERAGE,COVER_0_MAN,Trailing,1,44820
33844,2023122301,631,45571,Defensive Coverage,0.224575,0.566323,0.341748,4,C,17,2023,16,LAC,BUF,(5:48) J.Allen pass deep right to G.Davis to B...,1,05:48,CB,46076.0,11,MEDIUM,BUF,LAC,2nd_medium,2x2,GO,VERTICAL,17,INTERMEDIATE,TRADITIONAL,MAN_COVERAGE,COVER_1_MAN,Trailing,0,52536
22818,2023111910,1295,44903,Defensive Coverage,0.251261,0.590728,0.339467,3,I,0,2023,11,LA,SEA,(9:14) (Shotgun) G.Smith pass incomplete short...,2,09:14,SS,39987.0,9,QUICK,SEA,LA,2nd_long,2x2,OUT,OUT_BREAK,4,SHORT,TRADITIONAL,ZONE_COVERAGE,COVER_3_ZONE,Trailing,0,56471
36353,2023092411,3820,54708,Defensive Coverage,0.305356,0.640313,0.334957,5,C,9,2023,3,KC,CHI,(4:23) (Shotgun) J.Fields pass short left to D...,4,04:23,CB,53440.0,9,QUICK,CHI,KC,3rd_medium,2x2,SLANT,IN_BREAK,8,SHORT,TRADITIONAL,ZONE_COVERAGE,COVER_4_ZONE,Trailing,0,46093


In [77]:
# 2023091705	1530	53494	
# 2023100113	1796	
player_impacts[(player_impacts['game_id'] == 2023100113) 
               & (player_impacts['play_id'] == 1796)
              # & (player_impacts['nfl_id'] == 53494)
              ]

,game_id,play_id,nfl_id,player_role,baseline_prob,real_prob,delta,fold,pass_result,yards_gained,season,week,home_team_abbr,visitor_team_abbr,play_description,quarter,game_clock,player_position,passer_nfl_id,num_output_frames,num_output_frames_bin,possession_team,defensive_team,down_and_distance,receiver_alignment,route_of_targeted_receiver,route_type_of_targeted_receiver,pass_length,pass_length_bin,dropback_type,team_coverage_man_zone,team_coverage_type,defender_type,potential_contact,targeted_receiver_id
19703,2023100113,1796,46186,Defensive Coverage,0.780448,0.756839,-0.023609,3,C,15,2023,4,NYJ,KC,(3:47) (Shotgun) P.Mahomes pass short right to...,2,03:47,SS,44822.0,18,MEDIUM,KC,NYJ,3rd_medium,2x1,WHEEL,VERTICAL,14,INTERMEDIATE,TRADITIONAL,MAN_COVERAGE,COVER_1_MAN,Converging,0,53449
19704,2023100113,1796,52566,Defensive Coverage,0.480984,0.756839,0.275855,3,C,15,2023,4,NYJ,KC,(3:47) (Shotgun) P.Mahomes pass short right to...,2,03:47,CB,44822.0,18,MEDIUM,KC,NYJ,3rd_medium,2x1,WHEEL,VERTICAL,14,INTERMEDIATE,TRADITIONAL,MAN_COVERAGE,COVER_1_MAN,Trailing,1,53449
19705,2023100113,1796,53449,Targeted Receiver,0.902603,0.756839,-0.145764,3,C,15,2023,4,NYJ,KC,(3:47) (Shotgun) P.Mahomes pass short right to...,2,03:47,WR,44822.0,18,MEDIUM,KC,NYJ,3rd_medium,2x1,WHEEL,VERTICAL,14,INTERMEDIATE,TRADITIONAL,MAN_COVERAGE,COVER_1_MAN,None,0,53449
19706,2023100113,1796,54469,Defensive Coverage,0.756839,0.756839,0.000000,3,C,15,2023,4,NYJ,KC,(3:47) (Shotgun) P.Mahomes pass short right to...,2,03:47,CB,44822.0,18,MEDIUM,KC,NYJ,3rd_medium,2x1,WHEEL,VERTICAL,14,INTERMEDIATE,TRADITIONAL,MAN_COVERAGE,COVER_1_MAN,Converging,0,53449


In [78]:
# 2023091702	2483	52627
# 2023100810	2294	53554
# 2023120304	3059	52453
# 2023121701	1453	46501
# 2023091007	2208	52535
catch_probabilities[
  (catch_probabilities['game_id'] == 2023100113)&
  (catch_probabilities['play_id'] == 1796)
  # & (catch_probabilities['nfl_id'] == 1563)
  ]

,game_id,play_id,baseline_prob,real_prob,delta,pass_result,yards_gained,season,week,home_team_abbr,visitor_team_abbr,play_description,quarter,game_clock,passer_nfl_id,num_output_frames,num_output_frames_bin,possession_team,defensive_team,down_and_distance,receiver_alignment,route_of_targeted_receiver,route_type_of_targeted_receiver,pass_length,pass_length_bin,dropback_type,team_coverage_man_zone,team_coverage_type
5893,2023100113,1796,0.574298,0.756839,0.182541,C,15,2023,4,NYJ,KC,(3:47) (Shotgun) P.Mahomes pass short right to...,2,03:47,44822.0,18,MEDIUM,KC,NYJ,3rd_medium,2x1,WHEEL,VERTICAL,14,INTERMEDIATE,TRADITIONAL,MAN_COVERAGE,COVER_1_MAN


In [66]:
catch_probabilities.sort_values('delta', ascending = True).head(20)

,game_id,play_id,baseline_prob,real_prob,delta,pass_result,yards_gained,season,week,home_team_abbr,visitor_team_abbr,play_description,quarter,game_clock,passer_nfl_id,num_output_frames,num_output_frames_bin,possession_team,defensive_team,down_and_distance,receiver_alignment,route_of_targeted_receiver,route_type_of_targeted_receiver,pass_length,pass_length_bin,dropback_type,team_coverage_man_zone,team_coverage_type
7638,2024010703,1110,0.813640,0.094741,-0.718900,I,0,2023,18,DET,MIN,(13:43) (Shotgun) N.Mullens pass incomplete de...,2,13:43,45198.0,25,LONG,MIN,DET,1st_long,3x1,GO,VERTICAL,39,DEEP,TRADITIONAL,ZONE_COVERAGE,COVER_6_ZONE
12197,2023121701,3755,0.758428,0.150715,-0.607713,IN,0,2023,15,MIA,NYJ,(2:51) (Shotgun) T.Siemian pass deep left inte...,4,02:51,42593.0,29,LONG,NYJ,MIA,4th_down,3x1,CORNER,VERTICAL,29,DEEP,TRADITIONAL,MAN_COVERAGE,COVER_1_MAN
9639,2023120304,538,0.785533,0.195320,-0.590213,I,0,2023,13,TB,CAR,(7:38) (Shotgun) B.Mayfield pass incomplete sh...,1,07:38,46070.0,13,MEDIUM,TB,CAR,2nd_long,3x1,HITCH,STOP_HITCH,11,INTERMEDIATE,TRADITIONAL,ZONE_COVERAGE,COVER_3_ZONE
71,2023091002,3812,0.676094,0.099977,-0.576118,I,0,2023,1,CLE,CIN,(8:32) (Shotgun) J.Burrow pass incomplete deep...,4,08:32,52409.0,21,LONG,CIN,CLE,2nd_medium,2x2,GO,VERTICAL,24,DEEP,TRADITIONAL,MAN_COVERAGE,COVER_1_MAN
11422,2023110502,3507,0.761115,0.200823,-0.560293,I,0,2023,9,BAL,SEA,(9:49) (Shotgun) T.Huntley pass incomplete sho...,4,09:49,52895.0,18,MEDIUM,BAL,SEA,2nd_long,3x2,GO,VERTICAL,15,INTERMEDIATE,TRADITIONAL,ZONE_COVERAGE,COVER_3_ZONE
4077,2023111906,2507,0.713759,0.156910,-0.556849,IN,0,2023,11,MIA,LV,(8:22) (Shotgun) A.O'Connell pass deep middle ...,3,08:22,56000.0,12,MEDIUM,LV,MIA,1st_long,2x2,IN,IN_BREAK,16,INTERMEDIATE,TRADITIONAL,ZONE_COVERAGE,COVER_4_ZONE
3143,2023091709,1524,0.840888,0.296732,-0.544156,C,18,2023,2,LA,SF,(4:22) M.Stafford pass deep left to T.Atwell t...,2,04:22,34452.0,15,MEDIUM,LA,SF,1st_long,2x2,CORNER,VERTICAL,18,INTERMEDIATE,TRADITIONAL,ZONE_COVERAGE,COVER_6_ZONE
13127,2024010712,3704,0.741200,0.202810,-0.538389,C,34,2023,18,ARI,SEA,(2:00) (Shotgun) G.Smith pass deep right to T....,4,02:00,39987.0,24,LONG,SEA,ARI,1st_long,2x1,POST,VERTICAL,34,DEEP,TRADITIONAL,ZONE_COVERAGE,COVER_6_ZONE
4426,2023120302,2554,0.767228,0.243068,-0.524161,I,0,2023,13,NYJ,ATL,(11:18) T.Boyle pass deep right to G.Wilson pu...,3,11:18,46430.0,21,LONG,NYJ,ATL,2nd_long,2x2,GO,VERTICAL,21,DEEP,TRADITIONAL,ZONE_COVERAGE,COVER_6_ZONE
6033,2023100800,4503,0.802590,0.280553,-0.522037,C,29,2023,5,BUF,JAX,"(2:30) (No Huddle, Shotgun) J.Allen pass deep ...",4,02:30,46076.0,17,MEDIUM,BUF,JAX,1st_long,2x2,GO,VERTICAL,29,DEEP,SCRAMBLE,ZONE_COVERAGE,COVER_2_ZONE


In [ ]:
play

In [38]:
# 2023100810	2294	53554
# 2023120304	3059	52453
# 2023121701	1453	46501

local_submission[(local_submission['game_id'] == 2023091007) &
                 (local_submission['play_id'] == 2208) &
                 (local_submission['nfl_id'].isin([52535,54614]))
                 ]


,game_id,play_id,nfl_id,frame_id,pred_x,pred_y,actual_x,actual_y,error
16378,2023091007,2208,52535,1,38.010198,18.124799,38.02,18.13,0.011096
16379,2023091007,2208,52535,2,38.608961,18.196072,38.64,18.20,0.031286
16380,2023091007,2208,52535,3,39.235712,18.267338,39.30,18.27,0.064343
16381,2023091007,2208,52535,4,39.878581,18.343400,39.98,18.33,0.102300
16382,2023091007,2208,52535,5,40.532943,18.417998,40.69,18.39,0.159533
16383,2023091007,2208,52535,6,41.187157,18.495428,41.41,18.45,0.227427
16384,2023091007,2208,52535,7,41.851150,18.564567,42.14,18.48,0.300975
16385,2023091007,2208,52535,8,42.507415,18.633338,42.89,18.51,0.401974
16386,2023091007,2208,52535,9,43.151549,18.687268,43.65,18.52,0.525768
16387,2023091007,2208,52535,10,43.791309,18.743398,44.40,18.51,0.651904


In [39]:
# 2023100810	2294	53554
# 2023121701	1453	46501
train_input[(train_input['game_id'] == 2023091007) &
            (train_input['play_id'] == 2208) #&
            # (train_input['nfl_id']== 52535)
            ]

,game_id,play_id,player_to_predict,nfl_id,frame_id,play_direction,absolute_yardline_number,player_name,player_height,player_weight,player_birth_date,player_position,player_side,player_role,x,y,s,a,dir,o,num_frames_output,ball_land_x,ball_land_y
154104,2023091007,2208,False,43986,1,left,54,Antonio Hamilton,6-0,190,1993-01-24,CB,Defense,Defensive Coverage,53.40,32.66,0.02,0.03,38.50,96.40,10,45.169998,16.84
154105,2023091007,2208,False,43986,2,left,54,Antonio Hamilton,6-0,190,1993-01-24,CB,Defense,Defensive Coverage,53.39,32.69,0.04,0.64,1.01,110.00,10,45.169998,16.84
154106,2023091007,2208,False,43986,3,left,54,Antonio Hamilton,6-0,190,1993-01-24,CB,Defense,Defensive Coverage,53.39,32.71,0.18,1.66,344.54,115.51,10,45.169998,16.84
154107,2023091007,2208,False,43986,4,left,54,Antonio Hamilton,6-0,190,1993-01-24,CB,Defense,Defensive Coverage,53.37,32.75,0.49,2.67,341.18,117.62,10,45.169998,16.84
154108,2023091007,2208,False,43986,5,left,54,Antonio Hamilton,6-0,190,1993-01-24,CB,Defense,Defensive Coverage,53.34,32.83,0.99,3.77,337.74,102.60,10,45.169998,16.84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154505,2023091007,2208,True,54614,25,left,54,Cole Turner,6-6,240,2000-03-16,TE,Offense,Targeted Receiver,45.52,14.50,5.54,3.39,285.67,304.07,10,45.169998,16.84
154506,2023091007,2208,True,54614,26,left,54,Cole Turner,6-6,240,2000-03-16,TE,Offense,Targeted Receiver,45.00,14.66,5.28,4.32,290.91,321.84,10,45.169998,16.84
154507,2023091007,2208,True,54614,27,left,54,Cole Turner,6-6,240,2000-03-16,TE,Offense,Targeted Receiver,44.53,14.87,5.00,4.83,296.54,337.02,10,45.169998,16.84
154508,2023091007,2208,True,54614,28,left,54,Cole Turner,6-6,240,2000-03-16,TE,Offense,Targeted Receiver,44.11,15.11,4.70,5.10,303.14,352.30,10,45.169998,16.84


In [40]:
filtered[filtered['nfl_id'] == 45004].sort_values('delta', ascending= False).head(20)

,game_id,play_id,nfl_id,player_role,baseline_prob,real_prob,delta,fold,pass_result,yards_gained,season,week,home_team_abbr,visitor_team_abbr,play_description,quarter,game_clock,player_position,passer_nfl_id,num_output_frames,num_output_frames_bin,possession_team,defensive_team,down_and_distance,receiver_alignment,route_of_targeted_receiver,route_type_of_targeted_receiver,pass_length,pass_length_bin,dropback_type,team_coverage_man_zone,team_coverage_type,defender_type,potential_contact,targeted_receiver_id
7519,2023121709,2202,45004,Defensive Coverage,0.442949,0.620230,0.177280,1,I,0,2023,15,CAR,ATL,(11:37) D.Ridder pass incomplete deep right to...,3,11:37,FS,54539.0,14,MEDIUM,ATL,CAR,2nd_long,2x1,CORNER,VERTICAL,19,INTERMEDIATE,DESIGNED_ROLLOUT_RIGHT,ZONE_COVERAGE,COVER_6_ZONE,Converging,0,54473
43509,2024010705,519,45004,Defensive Coverage,0.663952,0.744226,0.080274,5,I,0,2023,18,CAR,TB,(5:56) (Shotgun) B.Mayfield pass incomplete de...,1,05:56,FS,46070.0,27,LONG,TB,CAR,3rd_medium,3x1,GO,VERTICAL,38,DEEP,TRADITIONAL,ZONE_COVERAGE,COVER_6_ZONE,Converging,0,41233
32427,2023120304,1700,45004,Defensive Coverage,0.367084,0.438982,0.071898,4,IN,0,2023,13,TB,CAR,(2:31) (Shotgun) B.Mayfield pass deep left int...,2,02:31,FS,46070.0,27,LONG,TB,CAR,3rd_short,3x1,GO,VERTICAL,28,DEEP,TRADITIONAL,ZONE_COVERAGE,COVER_3_ZONE,Converging,0,41233
35753,2023091800,1340,45004,Defensive Coverage,0.070475,0.115977,0.045502,5,I,0,2023,2,CAR,NO,"(11:03) (No Huddle, Shotgun) D.Carr pass incom...",2,11:03,FS,41265.0,23,LONG,NO,CAR,2nd_long,2x2,IN,IN_BREAK,23,DEEP,TRADITIONAL,ZONE_COVERAGE,COVER_3_ZONE,Converging,0,43336
30934,2023110900,1593,45004,Defensive Coverage,0.529044,0.574125,0.045082,4,C,15,2023,10,CHI,CAR,(5:18) (Shotgun) T.Bagent pass short middle to...,2,05:18,FS,56167.0,11,MEDIUM,CHI,CAR,4th_down,3x1,IN,IN_BREAK,15,INTERMEDIATE,TRADITIONAL,ZONE_COVERAGE,COVER_3_ZONE,Converging,0,55998
39874,2023112605,2665,45004,Defensive Coverage,0.084359,0.105622,0.021263,5,I,0,2023,12,TEN,CAR,(5:35) (Shotgun) W.Levis pass incomplete deep ...,3,05:35,FS,55898.0,28,LONG,TEN,CAR,3rd_long,3x1,GO,VERTICAL,30,DEEP,TRADITIONAL,ZONE_COVERAGE,COVER_3_ZONE,Converging,0,43396
39887,2023112605,2967,45004,Defensive Coverage,0.805037,0.817683,0.012647,5,C,19,2023,12,TEN,CAR,(:24) W.Levis pass deep left to D.Hopkins push...,3,00:24,FS,55898.0,15,MEDIUM,TEN,CAR,1st_long,2x1,OUT,OUT_BREAK,18,INTERMEDIATE,TRADITIONAL,ZONE_COVERAGE,COVER_6_ZONE,Converging,0,39973
32421,2023120304,1626,45004,Defensive Coverage,0.710987,0.720703,0.009716,4,C,13,2023,13,TB,CAR,(4:28) (Shotgun) B.Mayfield pass short middle ...,2,04:28,FS,46070.0,9,QUICK,TB,CAR,3rd_long,3x1,IN,IN_BREAK,13,INTERMEDIATE,TRADITIONAL,MAN_COVERAGE,COVER_1_MAN,Converging,0,41233
39814,2023112605,634,45004,Defensive Coverage,0.910981,0.913052,0.002071,5,C,25,2023,12,TEN,CAR,(1:56) W.Levis pass deep left to C.Okonkwo pus...,1,01:56,FS,55898.0,23,LONG,TEN,CAR,1st_long,2x1,WHEEL,VERTICAL,17,INTERMEDIATE,TRADITIONAL,ZONE_COVERAGE,COVER_6_ZONE,Converging,0,54608
35805,2023091800,2196,45004,Defensive Coverage,0.048958,0.050218,0.001260,5,I,0,2023,2,CAR,NO,(:49) (Shotgun) D.Carr pass incomplete deep le...,2,00:49,FS,41265.0,20,MEDIUM,NO,CAR,2nd_long,3x1,OUT,OUT_BREAK,23,DEEP,TRADITIONAL,ZONE_COVERAGE,COVER_6_ZONE,Converging,0,54476


In [41]:
denom = filtered.groupby('nfl_id').count().reset_index().sort_values('play_id', ascending = False)[['nfl_id','play_id']]
num = filtered[filtered['delta'] < -.01].groupby('nfl_id').count().reset_index().sort_values('play_id', ascending = False)[['nfl_id','play_id']]
f = num.merge(denom, on = 'nfl_id').rename(columns = {'play_id_x': 'num',
                                                      'play_id_y': 'denom'})
f['rate'] = f['num']/f['denom']
f.sort_values('num', ascending = False).head(1)


,nfl_id,num,denom,rate
0,53505,19,73,0.260274


In [42]:
test = pd.read_csv('./outputs/defender_impact_log_wr.csv')
print(test.shape)
test = test.merge(supplementary_data[['game_id','play_id','pass_result', 'yards_gained', 'season','week','home_team_abbr','visitor_team_abbr','play_description', 'quarter','game_clock']],
        on=['game_id','play_id'], how='left')
test = test.merge(train_input[['nfl_id','player_name']].drop_duplicates(), on='nfl_id', how='left')
test[test['nfl_id'] == 52535].sort_values('delta').head(2)

(46045, 8)


,game_id,play_id,nfl_id,player_role,baseline_prob,real_prob,delta,fold,pass_result,yards_gained,season,week,home_team_abbr,visitor_team_abbr,play_description,quarter,game_clock,player_name
36783,2023091007,2238,52535,Defensive Coverage,0.347514,0.278970,-0.068544,5,C,32,2023,1,WAS,ARI,(:14) (Shotgun) S.Howell pass deep left to C.S...,2,00:14,K'Von Wallace
24205,2023121703,3754,52535,Defensive Coverage,0.543693,0.479432,-0.064261,3,I,0,2023,15,TEN,HOU,(1:09) (Shotgun) C.Keenum pass incomplete shor...,4,01:09,K'Von Wallace
